# Miniproject 2

In [ ]:

!pip install backports.cached_property

# Setup matplotlib animation
import matplotlib
matplotlib.rc('animation', html='jshtml')


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
from typing import Optional


def implementation_for(cls, method_name: Optional[str] = None):
    """Helper to provide a concrete implementation for a class method.

    This utility allows one to define methods outside of a class defintion.
    Thus allowing one to provide the implementations of a class "incrementally". 
    This utility is useful for gradually submitting class method implementations 
    to catsoop. 

    It is intended to be used as a decorator.

    Args:
        cls: a Python class on which we are implementing the method.
        method_name: an optional method name to indicate which method we are 
            implementing. If not provided, infer the method name by using the 
            decorated function's name.

    Example:
        >>> class A: 
        ...   def foo(self):
        ...     raise NotImplementedError()

        >>> @implementation_for(A)
        ... def foo(self):
        ...   print("implemented foo!")

        >>> A().foo()
        implemented foo!
    """

    def decorator(meth: Callable) -> Callable:
        mname = method_name or meth.__name__
        setattr(cls, mname, meth)
        return meth

    return decorator



from typing import (Callable, Iterable, List, Sequence, Optional, Dict)

from abc import abstractmethod, ABCMeta
import itertools
import math
import random
import dataclasses
from collections import namedtuple
import copy

import numpy as np

import scipy.stats
import scipy.special

# Basic data structures.

# x, y coordinate and the rotation of the robot.
Pose = namedtuple("Pose", ["x", "y", "theta"])
# unique id (str) and x, y coordinate of the landmark.
Landmark = namedtuple("Landmark", ["id", "x", "y"])
# r --- distance to the landmark; b --- angle to the landmark, computed by arctan2.
Measurement = namedtuple("Measurement", ["landmark_id", "r", "b"])
# delta_p --- distance to move; delta_theta --- rotation of the robot
Command = namedtuple("Command", ["delta_p", "delta_theta"])


def grid_of_landmarks(
        x_range: Iterable[float] = range(-4, 20, 4),
        y_range: Iterable[float] = range(-4, 20, 4),
) -> Sequence[Landmark]:
    """Constructs a grid of landmarks from a catesion product of x and y coordinates."""
    return tuple(
        Landmark(f"landmark-{i}", *loc)
        for i, loc in enumerate(itertools.product(x_range, y_range)))


class Inference(metaclass=ABCMeta):
    """Interface for an inference algorithm (localization or SLAM variants)."""

    @abstractmethod
    def estimated_pose(self) -> Pose:
        """Returns the current estimation of the robot's Pose."""
        ...

    @abstractmethod
    def estimated_landmarks(self) -> Sequence[Landmark]:
        """Returns the current estimation of the landmarks."""
        ...

    @abstractmethod
    def init(self, init_state: Pose) -> None:
        """Initialize the inference with an initial pose."""
        ...

    @abstractmethod
    def update(self, measurements: Sequence[Measurement]) -> None:
        """An update step of the inference, called after every simulation step."""
        ...

    @abstractmethod
    def plot_state(self, ax) -> None:
        """Helper to visualize the internal state of the inference algorithm."""
        ...


def normalize_angles(angles: np.array) -> np.ndarray:
    """Given an array of angles in radians, element-wise normalize the angles to the range [-pi, pi].

    Args:
        angles: array of any shape.

    Returns:
        normalized angles, array of same shape as the input.
    """
    return np.arctan2(np.sin(angles), np.cos(angles))


def circular_mean(angles: np.array, axis=None) -> float:
    """Given an array of angles in radians, find the [circular mean](https://en.wikipedia.org/wiki/Circular_mean).

    Args:
        angles: array of any shape.
        axis: axis or axes along which the means are computed. The default is to compute the mean of the flattened array.

    Returns:
        mean of the angles, normalized to [-pi, pi].
    """
    return np.arctan2(np.sum(np.sin(angles), axis=axis),
                      np.sum(np.cos(angles), axis=axis))


@dataclasses.dataclass(frozen=True)
class SimulationResult:
    """A helper class to hold the results of running a simulation."""
    sim: 'Simulator'  # Simulator that generated this result
    infer: Optional[Inference]  # Inference procedure that generated this result
    true_poses: Sequence[Pose]
    estimated_poses: Optional[Sequence[Pose]] = None
    # History snapshots for the Inference, for animation purpose
    snapshots: Optional[Sequence[Inference]] = None


@dataclasses.dataclass(frozen=False)
class Simulator:
    """A simulator for a point robot in R^2.

    The simulator simulates the motion and sensor noises of the robot by 
    sampling from 2D Gaussians.
    The simulation has a list of landmarks, each with a unique identifier.
    The sensor of the robot can sense, for each landmark in the robot's sensing
    range, a noise-corrupted distance and bearing to that landmarks (together
    with the identifier that identifies the landmark).
    """

    # Initial simulator state
    init_state: Pose = Pose(0, 0, 0)

    # 2x2 covariance matrix of the motion noise.
    motion_noise_covariance: np.ndarray = np.diag([1e-3, np.deg2rad(3)**2])

    # 2x2 covariance matrix of the sensor noise.
    sensor_noise_covariance: np.ndarray = np.diag([1, np.deg2rad(5)**2])

    # A set of landmarks
    landmarks: Sequence[Landmark] = grid_of_landmarks()

    # Robot can only sense landmarks within this range
    max_sensing_range: float = np.inf

    # Current state of the robot
    state: Pose = dataclasses.field(init=False, default=None)

    # Random seed control
    rng: np.random.Generator = dataclasses.field(init=False, default=None)
    # Using the same seed will induce the same trajectory
    seed: dataclasses.InitVar[int] = 0

    def __post_init__(self, seed: int):
        assert (len({l.id for l in self.landmarks
                    }) == len(self.landmarks)), "Landmark must have unique IDs"
        self.rng = np.random.default_rng(seed)

    def init(self) -> None:
        self.state = self.init_state

    def simulate_motion(self, command: Command) -> None:
        """Simulate robot motion. This function updates the value of
        self.state.

        Args:
          command: a Command tuple containing fields delta_p(float), the 
          distance of the movement, and delta_theta(float), the rotation of 
          the movement.
        """
        noise = self.rng.multivariate_normal(np.zeros(2),
                                             self.motion_noise_covariance)
        x = (self.state.x + math.cos(self.state.theta) *
             (command.delta_p + noise[0]))
        y = (self.state.y + math.sin(self.state.theta) *
             (command.delta_p + noise[0]))
        theta = self.state.theta + command.delta_theta + noise[1]
        self.state = Pose(x, y, theta)

    def simulate_sensing(self) -> Sequence[Measurement]:
        """Simulate the robot sensing process. This function returns a list of
        measurements, in the same order as the self.landmarks list.
        Specifically, it measures the distance and the angle w.r.t. each
        landmark, and add a Gaussian noise on the measurement.

        Returns:
          measurements: a list of measurements.
        """
        measurements = []
        # Simulate that the measurements' order is random
        for landmark in sorted(self.landmarks, key=lambda _: random.random()):
            r = math.hypot(landmark.x - self.state.x, landmark.y - self.state.y)

            if r >= self.max_sensing_range:
                # Can't sense this landmark, skip
                continue

            b = math.atan2(landmark.y - self.state.y,
                           landmark.x - self.state.x) - self.state.theta
            # Normalize angle to -pi to pi
            b = math.atan2(math.sin(b), math.cos(b))

            noise = self.rng.multivariate_normal(np.zeros(2),
                                                 self.sensor_noise_covariance)

            measurements.append(
                Measurement(landmark.id, r + noise[0], b + noise[1]))

        return measurements

    def run(self,
            commands: Iterable[Command],
            infer: Optional[Inference] = None,
            snapshot_every_n: Optional[int] = 0) -> SimulationResult:
        """Run the simulation with a sequence of commands. Optionally, execute
        an inference algorithm along way with the simulation.

        Args:
            commands: an interable of commands.
            infer: the inference algorithm to run the simulation with.
            snapshot_every_n: snapshot the inference algorithm every n steps.
                if <= 0, don't take any snapshot. 

        Returns:
            SimulationResult
        """
        true_poses, est_poses, snapshots = [], None, None

        self.init()

        i = 0
        true_poses.append(self.state)
        if infer:
            est_poses, snapshots = [], []
            infer.init(self.init_state)
            est_poses.append(infer.estimated_pose())
            if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                snapshots.append(copy.deepcopy(infer))

        for command in commands:
            i += 1
            # Simulate motion
            self.simulate_motion(command)
            true_poses.append(self.state)
            if infer:
                # Simulate measurement
                measurements = self.simulate_sensing()
                infer.update(command, measurements)
                est_poses.append(infer.estimated_pose())
                if snapshot_every_n > 0 and i % snapshot_every_n == 0:
                    snapshots.append(copy.deepcopy(infer))

        return SimulationResult(self, copy.deepcopy(infer), true_poses,
                                est_poses, snapshots)


def drive_in_a_square_commands() -> Iterable[Command]:
    """Commands to simulate the robot's movement in a square."""
    for _ in range(4):
        for i in range(100):
            yield Command(.1, 0)
        yield Command(0, np.pi / 2)


def patrol_commands() -> Iterable[Command]:
    """Commands that walks back and forth horizontally."""
    for i in range(10):
        yield Command(0.5, 0)
    yield Command(0, np.pi)





def run_inference(sim: Simulator,
                  commands: Sequence[Command] = tuple(
                      drive_in_a_square_commands()),
                  mode: str = "localization",
                  num_particles: int = 100,
                  snapshot_every_n=0) -> SimulationResult:
    """Utility to run inference on a simulator with a particular inference mode.

    Args:
        sim: the simulator instance to run the simulation.
        commands: see `Simulator.run`.
        mode: specified the mode of the inference; one of "localization", "SLAM" or "RBSLAM".
            You must implement the corresponding inference algorithm in order 
            to perform inference in any mode.
        num_particles: number of particles of the particle filter.
        snapshot_every_n: see `Simulator.run`.

    Returns:
        A simulation result returned by `Simulator.run`.
    """

    mode = mode.lower()
    try:
        if mode == "localization":
            infer = Localization(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                landmarks=sim.landmarks,
                num_particles=num_particles,
            )
        elif mode == "slam":
            infer = SLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        elif mode == "rbslam":
            infer = RBSLAM(
                motion_noise_covariance=sim.motion_noise_covariance,
                sensor_noise_covariance=sim.sensor_noise_covariance,
                num_particles=num_particles,
            )
        else:
            raise ValueError(f"Unrecoginized inference: {mode}")
    except NotImplementedError:
        raise NotImplementedError(
            f"You must completely implement the {infer.__class__} class to "
            f"run simulation under {mode} mode")

    return sim.run(commands, infer=infer, snapshot_every_n=snapshot_every_n)


def plot_samples(x, y, fov=((-6, -6), (20, 20)), show=True):
    """Plots a list of samples of x, y coordinates as input and scatter plot the samples.

    Args:
        x: array of shape (N, ), the x-coordinates of the pose component of the particles.
        y: array of shape (N, ), the y-coordinates of the pose component of the particles.
        fov: a rectangular field of view. The function plots within this 
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(x, y, 'g+')

    if show:
        plt.show()

    return ax


def plot_trajectories(true_poses: Sequence[Pose],
                      estimated_poses: Sequence[Pose],
                      landmarks: Optional[Sequence[Landmark]] = None,
                      ax: Optional['matplotlib.axes.Axes'] = None,
                      fov=((-6, -6), (20, 20)),
                      show=True):
    """Plot true and estimated poses on the same plot by overlaying them.
    It ignores the pose.theta and only plots the location.

    Args:
        true_poses: a sequence of true poses.
        estimated_poses: a sequence of estimated poses.
        landmarks: optinally plot the location of true landmarks.
        ax: an optional matplotlib Axes on which to plot the trajectories.
        fov: a rectangular field of view. The function plots within this 
            rectanglular fov.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal', 'box')

    if landmarks:
        ax.scatter([l.x for l in landmarks], [l.y for l in landmarks],
                   marker="o",
                   color="black")

    ax.plot([p.x for p in true_poses], [p.y for p in true_poses],
            color=(0, 0, 1, 0.5))
    ax.plot([p.x for p in estimated_poses], [p.y for p in estimated_poses],
            color=(1, 0, 0, 0.5))

    if show:
        plt.show()
    return ax


def plot_simulation_result(result: SimulationResult,
                           ax: Optional['matplotlib.axes.Axes'] = None,
                           fov=((-6, -6), (20, 20)),
                           title=None,
                           show=True):
    """Plot the inference result (the state at last step of the simulation result).

    Args:
        result: result of a simulation.
        ax: an optional matplotlin Axes to draw the result.
        fov: see `plot_trajectories`.
        show: whether to display the plot immediately.
    """
    import matplotlib.pyplot as plt
    if ax is None:
        fig, ax = plt.subplots()

    if title:
        ax.set_title(title)

    plot_trajectories(result.true_poses,
                      result.estimated_poses,
                      landmarks=result.sim.landmarks,
                      ax=ax,
                      fov=fov,
                      show=False)

    result.infer.plot_state(ax)

    if show:
        plt.show()

    return ax


def render_animation(
    result: SimulationResult,
    fov=((-6, -6), (20, 20)),
    save_file: Optional[str] = None,
) -> 'matplotlib.animation.FuncAnimation':
    """Render simulation as a matplotlib animation.

    Args:
        result: result of a simulation.
        fov: see `plot_trajectories`.
        save_file: an optional string file name to save the animation. 

    Returns:
        an instance of `matplotlib.animation.FuncAnimation`. You can do further 
        processing on the returned animation, display it on Colab, or save it
        as an mp4 file.  
    """
    if result.snapshots is None:
        raise ValueError("SimulationResult does not have any snapshots. "
                         "Please re-run simulation with snapshot_every_n > 0.")

    import matplotlib.animation as animation
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    snapshot_every_n = len(result.true_poses) // len(result.snapshots)
    true_poses = result.true_poses[::snapshot_every_n]
    estimated_poses = result.estimated_poses[::snapshot_every_n]

    def render_frame(i):
        ax.clear()
        ax.set_title(f"Step {i}")
        plot_trajectories(true_poses[:i + 1],
                          estimated_poses[:i + 1],
                          landmarks=result.sim.landmarks,
                          ax=ax,
                          fov=fov,
                          show=False)
        result.snapshots[i].plot_state(ax)
        return ax

    anim = animation.FuncAnimation(fig, render_frame, len(result.snapshots))
    if save_file:
        anim.save(f"{save_file}.mp4", fps=60)
    return anim





## Warmup 1


### Question
Let's make sure we know how to sample from a set 
of values based on their weights.

Please write a function that:
- Takes in a list of samples and associated weights.
- Re-samples from the list of samples with replacement according the
likelihood of each sample given by the list of weights.

_Hint:_ You should use the function `numpy.random.choice`.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_1(samples: np.ndarray, weights: np.ndarray,
             nr_samples: int) -> np.ndarray:
    """Draw N samples from the input list based on their weights.

    Args:
      samples: a numpy array of shape (n_samples, ), a list of samples.
      weights: a numpy float array of shape (n_samples, ), indicating 
        the weights for individual samples.
      nr_samples: an integer, indicating the number of samples to draw.

    Returns:
      resampled_samples: the return numpy array, containing nr_samples
        samples drawn from the input list, based on the weights with replacement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_1(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=np.int64), np.array([0.03191091, 0.10365558, 0.07293406, 0.13085621, 0.12995933, 0.0454193 , 0.04606439, 0.1336077 , 0.15964489, 0.14594763], dtype=np.float64), 10), np.array([6, 8, 7, 6, 4, 7, 4, 9, 9, 4], dtype=np.int64), atol=1e-6)
print('Tests passed.')

## Warmup 2


### Question
We are going to be sampling from Gaussian distributions in this miniproject. 
First, let us check that we know how to sample from a multivariate Gaussian.
Please write a function that takes in a mean vector, a covariance matrix and a integer number 
of samples and returns a list of the requested number of samples drawn from a Gaussian with the given mean and covariance.

_Hint:_ You should use the function `numpy.random.multivariate_normal`.


For reference, our solution is **2** line(s) of code.

In [ ]:
def warmup_2(mean: np.ndarray, covariance: np.ndarray,
             num_particles: int) -> np.ndarray:
    """Sample from a multi-variate Gaussian distribution.

    Args:
      mean: a numpy array of shape (N, ).
      covariance: a numpy array of shape (N, N).
      num_particles: an integer.

    Returns:
      sample: a numpy vector with length num_particles.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
import random; import numpy.random; random.seed(0); numpy.random.seed(0);
assert np.allclose(warmup_2(np.array([0.58345008, 0.05778984, 0.13608912], dtype=np.float64), np.array([[1.61405906, 0.93399482, 0.79165733],  [0.93399482, 1.65599579, 1.43616931],  [0.79165733, 1.43616931, 1.25177385]], dtype=np.float64), 5), np.array([[-0.8347847 , -2.30256418, -1.83805061],  [-0.13115218, -3.33917428, -2.91354857],  [-0.47476854, -1.05885092, -0.83435982],  [ 0.29510331, -0.55660856, -0.28675913],  [-0.06919861, -0.94320128, -0.69794801]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## Localization: Update by Motion Model


### Utilities

Particle Filter for Localization
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

try:
    from functools import cached_property
except ImportError:
    # Import for Colab (Python==3.7)
    from backports.cached_property import cached_property


def multivariate_normal_logpdf(xs: np.ndarray, means: np.ndarray,
                               covs: np.ndarray) -> np.ndarray:
    """Batch version of `scipy.stats.multivariate_normal.logpdf`.

    Compute multivariate normal's log probability density function, with batch 
    inputs and distributions.
    Modified from https://gregorygundersen.com/blog/2020/12/12/group-multivariate-normal-pdf/. 

    This function uses the following broadcasting logic:
    - First, `means` and `covs` are broadcasted together, which determines `dist_batch`, 
        the number of individual multivariate normal distributions. 
    - Then, broadcast the rightmost dimensions of `xs` to `means`. The rest of the dimensions 
    of `xs` becomes the `input_batch` dimension.

    Args:
        xs: array of shape (...input_batch,  ...dist_batch, dist_dim)
        means: array of shape (...dist_batch, dist_dim)
        covs: array of shape (...dist_batch, dist_dim, dist_dim)

    Returns:
        array of shape (...input_batch, ...dist_batch, dist_dim)


    Below we describe some uses cases, with different input shapes and the result shape.

    1.  xs: (100, 2), means: (2,),  covs: (2, 2), result: (100, 2)
        Explanation: 
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=(100,)` and `dist_batch=()`
            - returns logpdf of 100 inputs, evaluated by the same mean and covariance.

    2.  xs: (100, 2), means: (2,),  covs: (100, 2, 2), result: (100, 2)
        Explanation: 
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100,)`
            - returns logpdf of 100 inputs, evaluated by the same mean but with 100 different covariances.

    3.  xs: (100, 10, 2), means: (2,),  covs: (100, 10, 2, 2), result: (100, 10, 2)
        Explanation: 
            - dist_dim = 2 i.e., Gaussian is 2D
            - `input_batch=()` and `dist_batch=(100, 10)`
            - returns logpdf of inputs of shape (100, 10), evaluated by the same mean each with a different covariance.
    """
    dist_dim = xs.shape[-1]
    dist_batch_shape = np.broadcast_shapes(means.shape[:-1], covs.shape[:-2])
    means = np.broadcast_to(means, dist_batch_shape + (dist_dim,))
    covs = np.broadcast_to(covs, dist_batch_shape + (dist_dim, dist_dim))
    xs = np.broadcast_to(xs, xs.shape[:-len(means.shape)] + means.shape)

    vals, vecs = np.linalg.eigh(covs)
    logdets = np.sum(np.log(vals), axis=-1)
    valsinvs = 1. / vals
    Us = vecs * np.sqrt(valsinvs)[..., np.newaxis, :]
    devs = xs - means
    devUs = np.einsum("...i,...ij->...j", devs, Us)
    mahas = np.sum(np.square(devUs), axis=-1)
    log2pi = np.log(2 * np.pi)
    out = -0.5 * (dist_dim * log2pi + mahas + logdets)
    return out


@dataclasses.dataclass(frozen=True)
class LocalizationParticles:
    """A batch of particles."""

    x: np.ndarray  # shape (N, )
    y: np.ndarray  # shape (N, )
    theta: np.ndarray  # shape (N, )

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert len(self.x.shape) == 1
        # Always normalize the angles
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class Localization(Inference):
    """Localization Inference by particle filtering."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # In localization, we assume that we have access to the set of landmarks.
    landmarks: List[Landmark] = dataclasses.field(default_factory=list)

    num_particles: int = 10

    particles: LocalizationParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    @cached_property
    def landmarks_id_map(self) -> Dict[str, Landmark]:
        """Map from unique landmark id to the Landmark instance."""
        return {l.id: l for l in self.landmarks}

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        # we know the landmarks so no estimation required!
        return self.landmarks

    def init(self, init_state: Pose) -> None:
        self.particles = LocalizationParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.4),
        )

    # We incrementally implement methods below using the `implementation_for` helper
    def motion_model(self, command: Command):
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command, measurements: Sequence[Measurement]):
        raise NotImplementedError()


def plot_particles(samples: LocalizationParticles, fov=((-6, -6), (20, 20))):
    """matplotlib helper function. It takes a list of samples as input and
    scatter plot the samples.

    Args:
        particles: a set of particles for localization. This function only plots 
            the first two components: x and y.
        fov: a rectangular field of view. The function plots within this 
            rectanglular fov.
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()

    ax.axis([fov[0][0], fov[1][0], fov[0][1], fov[1][1]])

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.plot(samples.x, samples.y, 'g+')
    plt.show()

### Question

Let us now start by building a motion model for a
simple ground robot that moves in straight lines, and performs point turns.

A common motion model for such a robot assumes that the robot has a pose given
by $[x_t, y_t, \theta_t]$ at time $t$. The robot takes a command $[\Delta p_t,
\Delta \theta_t]$ which modifies the robot's pose according to the following:

$$
\begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \end{bmatrix} =
\begin{bmatrix} x_t + \cos(\theta_t) (\Delta p_t + \omega_t) \\
y_t + \sin(\theta_t)(\Delta p_t + \omega_t) \\
\theta_t + \Delta \theta_t + \nu_t
\end{bmatrix}
$$

where $[\omega_t, \nu_t]$ is Gaussian error injected at each time
step, distributed as $\omega_t \sim \mathcal{N}(0, \sigma^2_p)$, and
$\nu_t \sim \mathcal{N}(0, \sigma^2_\theta)$. The units of $\Delta  p_t$ is in
metres and $\Delta \theta_t$ in radians.

Let us assume that the robot's pose at each time step is a random variable,
represented as a set of samples.

Please implement a function that takes in a set of sampled poses
drawn from prior pose random variable $\mathbf{X}_t$
and a control $[\Delta p_t, \Delta \theta_t]$ and returns a set
of samples drawn from the posterior $X_{t+1}$.

**Note**
In this project, we will use a helper decorator `implementation_for` to implement
class methods outside of the class. 
The decorator `@implementation_for(Localization)` simply inserts the declared
function as a method of the Localization class. 
Using this decorator, you copy only the implemented method into 
catsoop's solution box and avoid repeating other parts of the class definition.


**Hint**: Use `np.random.multivariate_normal` to generate the noise.


For reference, our solution is **10** line(s) of code.

In [ ]:
@implementation_for(Localization)
def motion_model(self: Localization, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot, and
    updates to the new samples.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def localization_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)))
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = Localization(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        landmarks=sim.landmarks,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

localization_motion_model_problem_test()

print('Tests passed.')

## Localization: Computing the Importance Weights


### Question

Let us now add in a measurement model. Let us assume that
there are four landmarks located at $(x, y)$ locations (-1, -1), (11, -1), (11, 11) and
(-1, 11). The robot is equipped with a sensor that can measure the range and
relative bearing $[r, b]$ from the robot's location to each of these landmarks. If the
robot's pose is $(x, y, \theta)$ and the landmark position is $(l_x, l_y)$,
then range is just the Euclidean distance $$r = \sqrt{(l_x - x)^2 + (l_y
-y)^2}$$, 
and the bearing is the angle to the landmark in the robot's body frame (hence
relative bearing), $$b = \texttt{atan2}(l_y - y, l_x - x) - \theta.$$  Each of
these measurements is corrupted with noise $[q_r, q_b]$ distributed according
to $q_r \sim \mathcal{N}(0, \sigma^2_r)$ and $q_b \sim \mathcal{N}(0,
\sigma^2_b)$.

Please take a look at our implementation in `simulator.simulate_sensing` and
understand how the values are simulated.

Now the problem is that for a real robot, even if you know this is the model
that generates the measurements, you can't know the noise that nature applied. (That would be omniscience!) 
What we can do is work out how likely each measurement is. More precisely, given a set of measurements, we can use
this model to compute the importance weights of a set of samples of the robot
pose. See [Lecture 13 Page 13](https://sicp-s4.mit.edu/_static/fall22/lectures/lec13/L13-Sampling.pdf#page=13)
for details.

In particular, to compute the importance weight of a particle:
- Compute the _predicted measurements_ from the current state of the robot's pose and the landmark locations.
- Compute the errors $\mathit{err}$ between the predicted and true measurements.
- Compute the likelihood of the $\mathit{err}$ as the unnormalized importance weights. 
The likelihood of the error is the evaluation of the probability density function (PDF) of the two-dimensional Gaussian distribution with covariance `sensor_noise_covariance` at $\mathit{err}$.
- Finally, normalize the weights so that they sum to one. 

Please implement a function that takes a set of samples and measurements and
returns the importance weights of the samples.

There are a few technicalities during this process:
- We highly recommend you normalize **all the rotational values** (in radians) to the interval $[-\pi, \pi]$. 
Notably, the rotational component of the $\mathit{err}$, which we denote as $\mathit{err}_\theta$, is a rotational 
value in radians. Therefore, theoretically, the likelihood of $\mathit{err}_\theta$ should have been
the evaluation of the PDF of a [wrapped Gaussian distribution](https://en.wikipedia.org/wiki/Wrapped_normal_distribution). 
In other words, to evaluate the likelihood of 
$\mathit{err}_\theta$, we should have taken the sum of the Gaussian PDFs at the points 
$\mathit{err}_\theta + 2k\pi$ for all $k \in \mathbb{Z}$. 
Nonetheless, since our sensor noise is relatively small, we expect only the single point at $k=0$
$\mathit{err}_\theta \in [-\pi, \pi]$ to have a relatively large value, and all other
points to have close-to-zero values. Therefore, we may use the Gaussian distribution as a 
good approximation to the true wrapped Gaussian distribution.
- The likelihood for the particles can become really small. For numerical stability, 
you will want to compute the _log-likelihood_ instead of the likelihood. 
To that end, you may:
    - Use the utility `multivariate_normal_logpdf` that we provided you to evaluate the log PDF. 
    - Take the sum (instead of multiplications) of the log-likelihoods to compute the unnormalized log-weight of each particle.
    - Normalize the log-weights. You should use [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html) to evaluate 
the log-sum of the weights. In short, The `logsumexp` function calculates the following expression in a more 
numerically stable way:
$$
\texttt{logsumexp}(x_1, ... x_n) = \log \left( \sum_{i=1}^{n} \exp(x_i) \right).
$$ See <a href="https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/" target="_blank">this blog</a> for a more detailed description of this function.
    - Use `numpy.exp` to undo the logarithm from the log-weights and return the weights.


For reference, our solution is **29** line(s) of code.

In [ ]:
@implementation_for(Localization)
def compute_weights(self: Localization,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the samples, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def localization_compute_weights_test(_, infer: Localization,
                                      measurements: Sequence[Measurement],
                                      result_weights: np.ndarray):
    weights = infer.compute_weights(measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

localization_compute_weights_test(compute_weights, Localization(motion_noise_covariance=np.array([[0.001     , 0.        ],  [0.        , 0.00274156]], dtype=np.float64), sensor_noise_covariance=np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), landmarks=(Landmark(id='landmark-0', x=-4, y=-4), Landmark(id='landmark-1', x=-4, y=0), Landmark(id='landmark-2', x=-4, y=4), Landmark(id='landmark-3', x=-4, y=8), Landmark(id='landmark-4', x=-4, y=12), Landmark(id='landmark-5', x=-4, y=16), Landmark(id='landmark-6', x=0, y=-4), Landmark(id='landmark-7', x=0, y=0), Landmark(id='landmark-8', x=0, y=4), Landmark(id='landmark-9', x=0, y=8), Landmark(id='landmark-10', x=0, y=12), Landmark(id='landmark-11', x=0, y=16), Landmark(id='landmark-12', x=4, y=-4), Landmark(id='landmark-13', x=4, y=0), Landmark(id='landmark-14', x=4, y=4), Landmark(id='landmark-15', x=4, y=8), Landmark(id='landmark-16', x=4, y=12), Landmark(id='landmark-17', x=4, y=16), Landmark(id='landmark-18', x=8, y=-4), Landmark(id='landmark-19', x=8, y=0), Landmark(id='landmark-20', x=8, y=4), Landmark(id='landmark-21', x=8, y=8), Landmark(id='landmark-22', x=8, y=12), Landmark(id='landmark-23', x=8, y=16), Landmark(id='landmark-24', x=12, y=-4), Landmark(id='landmark-25', x=12, y=0), Landmark(id='landmark-26', x=12, y=4), Landmark(id='landmark-27', x=12, y=8), Landmark(id='landmark-28', x=12, y=12), Landmark(id='landmark-29', x=12, y=16), Landmark(id='landmark-30', x=16, y=-4), Landmark(id='landmark-31', x=16, y=0), Landmark(id='landmark-32', x=16, y=4), Landmark(id='landmark-33', x=16, y=8), Landmark(id='landmark-34', x=16, y=12), Landmark(id='landmark-35', x=16, y=16)), num_particles=20, particles=LocalizationParticles(x=np.zeros((20,), dtype=np.float64), y=np.zeros((20,), dtype=np.float64), theta=np.zeros((20,), dtype=np.float64))), [Measurement(landmark_id='landmark-35', r=22.753147219062917, b=0.7738698392857971), Measurement(landmark_id='landmark-25', r=12.640422650443282, b=0.009154262150241616), Measurement(landmark_id='landmark-15', r=8.408602536838048, b=1.1387038391295914), Measurement(landmark_id='landmark-3', r=10.248271955129297, b=2.117092341243524), Measurement(landmark_id='landmark-32', r=15.78868726666365, b=0.1345498076509456), Measurement(landmark_id='landmark-12', r=5.033579786955029, b=-0.7817917913663452), Measurement(landmark_id='landmark-7', r=-2.3250307746388343, b=-0.01909317455769924), Measurement(landmark_id='landmark-20', r=7.698360962746094, b=0.399745227278584), Measurement(landmark_id='landmark-5', r=15.948163519613333, b=1.7881725941556874), Measurement(landmark_id='landmark-2', r=6.068484785866513, b=2.4471709441565186), Measurement(landmark_id='landmark-26', r=12.520575977729484, b=0.44099698774324514), Measurement(landmark_id='landmark-24', r=11.983915967186904, b=-0.291075514011446), Measurement(landmark_id='landmark-8', r=4.903470181651809, b=1.5790004474613593), Measurement(landmark_id='landmark-30', r=15.748923253116834, b=-0.32541437620132907), Measurement(landmark_id='landmark-11', r=15.542274174332661, b=1.5900119763018357), Measurement(landmark_id='landmark-4', r=11.639492457134782, b=1.8742928687102223), Measurement(landmark_id='landmark-34', r=19.84077499008552, b=0.6906987897814902), Measurement(landmark_id='landmark-9', r=8.21465912250634, b=1.6018084460172888), Measurement(landmark_id='landmark-27', r=13.76837649243762, b=0.5766916802305041), Measurement(landmark_id='landmark-14', r=6.44082971955371, b=0.9157246165771075), Measurement(landmark_id='landmark-23', r=16.629478287894198, b=1.2392634902611783), Measurement(landmark_id='landmark-21', r=12.659583922767066, b=0.8535804455254041), Measurement(landmark_id='landmark-13', r=4.264455630329303, b=-0.027394933553940907), Measurement(landmark_id='landmark-1', r=5.458020683536959, b=3.312657462648057), Measurement(landmark_id='landmark-6', r=5.801634869866125, b=-1.4560318732949584), Measurement(landmark_id='landmark-33', r=18.245924230657273, b=0.3582019162682134), Measurement(landmark_id='landmark-18', r=8.939817776879076, b=-0.40635936363131575), Measurement(landmark_id='landmark-0', r=4.368492785742826, b=-2.3217135857051487), Measurement(landmark_id='landmark-31', r=16.42986369482223, b=0.06074118633851044), Measurement(landmark_id='landmark-22', r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id='landmark-19', r=7.563564752856779, b=-0.10208447443373762), Measurement(landmark_id='landmark-10', r=13.739367877130134, b=1.5275199517585014), Measurement(landmark_id='landmark-16', r=12.97808027013372, b=1.2264810610293408), Measurement(landmark_id='landmark-28', r=18.554035627279266, b=0.9006213960947502), Measurement(landmark_id='landmark-29', r=20.633352622824916, b=0.7350027054140555), Measurement(landmark_id='landmark-17', r=16.54445147673053, b=1.3854805390653757)], np.full((20,), 0.05000000000000003, dtype=np.float64))
print('Tests passed.')

## Localization: Particle Filter Update


### Question

What we want you to do is to write a method for step 4: `update` that takes in a
set of samples, the most recent command, and the most recent measurements, and then:
  * Use the motion_model method to update the sample based on the command
  * Use compute_weights to compute the sample weights from the
measurement,
  * Resample a new set of samples,
  * Blur the samples by applying noise drawn from a zero-mean Gaussian with
covariance equal to `np.diag([1e-3, 1e-3, np.deg2rad(1)**2])`. (You'll need to
recall from lecture why we do this blurring --- if you have machine learning
background, it's a form of regularization, but you don't need to know about
regularization to understand why this is important.)


For reference, our solution is **16** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(Localization)
def update(self: Localization, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def localization_update_test(_, infer: Localization, command: Command,
                             measurements: Sequence[Measurement],
                             pose: Pose):
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.3), "Estimated pose should be quite close to our estimation"

localization_update_test(update, Localization(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 9.27917724e-08]], dtype=np.float64), landmarks=(Landmark(id='landmark-0', x=-4, y=-4), Landmark(id='landmark-1', x=-4, y=0), Landmark(id='landmark-2', x=-4, y=4), Landmark(id='landmark-3', x=-4, y=8), Landmark(id='landmark-4', x=-4, y=12), Landmark(id='landmark-5', x=-4, y=16), Landmark(id='landmark-6', x=0, y=-4), Landmark(id='landmark-7', x=0, y=0), Landmark(id='landmark-8', x=0, y=4), Landmark(id='landmark-9', x=0, y=8), Landmark(id='landmark-10', x=0, y=12), Landmark(id='landmark-11', x=0, y=16), Landmark(id='landmark-12', x=4, y=-4), Landmark(id='landmark-13', x=4, y=0), Landmark(id='landmark-14', x=4, y=4), Landmark(id='landmark-15', x=4, y=8), Landmark(id='landmark-16', x=4, y=12), Landmark(id='landmark-17', x=4, y=16), Landmark(id='landmark-18', x=8, y=-4), Landmark(id='landmark-19', x=8, y=0), Landmark(id='landmark-20', x=8, y=4), Landmark(id='landmark-21', x=8, y=8), Landmark(id='landmark-22', x=8, y=12), Landmark(id='landmark-23', x=8, y=16), Landmark(id='landmark-24', x=12, y=-4), Landmark(id='landmark-25', x=12, y=0), Landmark(id='landmark-26', x=12, y=4), Landmark(id='landmark-27', x=12, y=8), Landmark(id='landmark-28', x=12, y=12), Landmark(id='landmark-29', x=12, y=16), Landmark(id='landmark-30', x=16, y=-4), Landmark(id='landmark-31', x=16, y=0), Landmark(id='landmark-32', x=16, y=4), Landmark(id='landmark-33', x=16, y=8), Landmark(id='landmark-34', x=16, y=12), Landmark(id='landmark-35', x=16, y=16)), num_particles=100, particles=LocalizationParticles(x=np.zeros((100,), dtype=np.float64), y=np.zeros((100,), dtype=np.float64), theta=np.zeros((100,), dtype=np.float64))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id='landmark-4', r=12.649236370894611, b=1.8925066397489418), Measurement(landmark_id='landmark-16', r=12.649751063323961, b=1.249077726801277), Measurement(landmark_id='landmark-20', r=8.943736240625999, b=0.46375775715344025), Measurement(landmark_id='landmark-35', r=22.628720998014654, b=0.7856866607567657), Measurement(landmark_id='landmark-34', r=19.999296264764194, b=0.6431156393698316), Measurement(landmark_id='landmark-10', r=11.999376725537463, b=1.5708089154080955), Measurement(landmark_id='landmark-12', r=5.654529218717742, b=-0.7854648111495861), Measurement(landmark_id='landmark-7', r=-0.0012459109472530653, b=-0.00022306139218380394), Measurement(landmark_id='landmark-33', r=17.887999561015462, b=0.4635512584632948), Measurement(landmark_id='landmark-17', r=16.492834133007015, b=1.3261352314007253), Measurement(landmark_id='landmark-8', r=3.999871465337056, b=1.5712125753715283), Measurement(landmark_id='landmark-31', r=15.999334805326514, b=0.00010707609058078074), Measurement(landmark_id='landmark-1', r=4.000903470181652, b=3.141621291373365), Measurement(landmark_id='landmark-29', r=19.999256500750647, b=0.9270144443957444), Measurement(landmark_id='landmark-15', r=8.943814184173492, b=1.1072157930644515), Measurement(landmark_id='landmark-5', r=16.491412884287104, b=1.815711271397861), Measurement(landmark_id='landmark-18', r=8.944112684989244, b=-0.46348285801429606), Measurement(landmark_id='landmark-25', r=12.000214659122506, b=0.00010825271769035358), Measurement(landmark_id='landmark-23', r=17.8878899913889, b=1.1071092352234257), Measurement(landmark_id='landmark-21', r=11.314492474454822, b=0.7858530885395344), Measurement(landmark_id='landmark-11', r=15.998740934467897, b=1.5712574943489113), Measurement(landmark_id='landmark-32', r=16.493768377894426, b=0.24521666418979546), Measurement(landmark_id='landmark-28', r=16.970827204107472, b=0.7853025370396719), Measurement(landmark_id='landmark-30', r=16.49388052315418, b=-0.24438153429639173), Measurement(landmark_id='landmark-27', r=14.424006736725824, b=0.5884032070631326), Measurement(landmark_id='landmark-3', r=8.944629290409818, b=2.0340758608916567), Measurement(landmark_id='landmark-22', r=14.422200647722839, b=0.9829936969482067), Measurement(landmark_id='landmark-0', r=5.655565888028631, b=-2.3560741291298712), Measurement(landmark_id='landmark-14', r=5.6572841131872025, b=0.785610190136083), Measurement(landmark_id='landmark-24', r=12.647926522706761, b=-0.32195212052680294), Measurement(landmark_id='landmark-2', r=5.656417814245238, b=2.3558381481535373), Measurement(landmark_id='landmark-6', r=4.00173936787713, b=-1.5709473898414392), Measurement(landmark_id='landmark-9', r=8.00032896962946, b=1.5707175610932667), Measurement(landmark_id='landmark-19', r=8.001583472878803, b=0.00040220495707190147), Measurement(landmark_id='landmark-26', r=12.649743993296344, b=0.3210793269023251), Measurement(landmark_id='landmark-13', r=4.00005202897426, b=0.00020826272337815132)], Pose(x=0.9858299248258179, y=-0.00020686081759629056, theta=0.7682645354557489))
print('Tests passed.')

## SLAM: Update by Motion Model


### Utilities

SLAM Particle Filter
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


@dataclasses.dataclass(frozen=True)
class SLAMParticles:
    """A batch of samples for SLAM."""
    x: np.ndarray  # shape: (num_particles, )
    y: np.ndarray  # shape: (num_particles, )
    theta: np.ndarray  # shape: (num_particles, )

    # Mapping from landmark_id to the index of that landmark in `landmarks_loc`
    landmarks_id_to_idx: Dict[str, int]

    landmarks_loc: np.ndarray  # shape (num_particles, num_landmarks, 2)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape, "shape must match"
        assert self.x.shape[0] == self.landmarks_loc.shape[
            0], "shape must match"
        assert self.landmarks_loc.shape[1] == len(
            self.landmarks_id_to_idx), "shape must match"
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of samples."""
        return self.x.shape[0]


@dataclasses.dataclass
class SLAM(Inference):

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # A 2x2 array for the covariance when initializing new landmarks
    new_landmarks_init_covariance: np.ndarray = np.eye(2) * 10

    num_particles: int = 10

    particles: SLAMParticles = None

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        return [
            Landmark(idx_to_id[i], *loc) for i, loc in enumerate(landmarks_loc)
        ]

    def init(self, init_state: Pose) -> None:
        self.particles = SLAMParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
            {},
            np.empty((self.num_particles, 0, 2), dtype=np.float64),
        )

    def get_measured_landmarks_idx(
            self, measurements: Sequence[Measurement]) -> np.ndarray:
        """Given a sequence of measurements, find the indices of each of the l
        andmark in `self.particles`.

        Args:
            measurements: a sequence of measures. This function assumes that all
            landmarks in this input sequence of measurements are already tracked 
            (i.e., using `self.add_new_landmarks`).
        """
        return np.array(
            [
                self.particles.landmarks_id_to_idx[m.landmark_id]
                for m in measurements
            ],
            dtype=int,
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        import matplotlib.pyplot as plt
        cmap = plt.get_cmap('viridis')

        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )

        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        landmark_idx_to_color_code = np.array([
            float(hash(idx_to_id[idx]) % 256) / 256
            for idx in range(self.particles.landmarks_loc.shape[1])
        ])

        ax.scatter(self.particles.landmarks_loc[..., 0].flatten(),
                   self.particles.landmarks_loc[..., 1].flatten(),
                   marker="x",
                   c=np.broadcast_to(
                       landmark_idx_to_color_code,
                       self.particles.landmarks_loc.shape[:-1]).flatten(),
                   cmap=cmap)

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> None:
        raise NotImplementedError()

    def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def compute_weights(self,
                        measurements: Sequence[Measurement]) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

### Question

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$ 
and updates the particles by drawing from the posterior $X_{t+1}$ using the motion model. 

This function behaves the same as `Localization.motion_model` for the pose 
components of the particles, and simply keeps the landmarks components 
(`SLAMParticles.landmarks_id_to_idx` and `SLAMParticles.landmarks_loc`) 
of the particles the same.

**Hint**

Please take a look at the fields of the `SLAMParticles` class.
Note that `landmarks_id_to_idx` stores all the landmarks the robot has encountered so far.
It is a mapping from a landmark identifier (a string) to an index (an integer). 
You can use the integer index to index into the `landmarks_loc` array. 
In particular, `landmarks_loc` is an array of shape `(num_particles, len(landmarks_id_to_idx), 2)`.
Therefore, to access the coordinate of the landmark with the identifier `"Stata"` contained
in the $i$-th particle:
```python
# assert isinstance(particles, SLAMParticles)
loc = particles.landmarks_loc[i, particles.landmarks_id_to_idx["Stata"]]
```


For reference, our solution is **12** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def motion_model(self: SLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot and updates the particles.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def slam_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)),
                    landmarks=[])
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = SLAM(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

slam_motion_model_problem_test()

print('Tests passed.')

## SLAM: Tracking New Landmarks


### Question

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we have to match newly observed landmarks to ones we have seen before, 
and add landmarks that we are seeing for the first time to our map.  
In some situations, the matching is difficult, but for our purposes, 
we’ll assume that the landmarks are uniquely identifiable, so we know exactly 
which ones we are observing on each step.

One way to handle missing data due to sensing range is to explicitly model the 
sensing range in our probabilistic model of the sensors. 
Howewer, this approach would induce a difficult inference problem.  
Instead, we will take the approach of treating the missingness as uninformative.
In particular, we will consider sensor data within-range as observed variable, 
and sensor data out-of-range as unobserved variables.
When we see a new landmark for the first time, we will assume that our belief 
about the location of that landmark has a mean equal to that first observed location 
and a standard variance (new_landmarks_init_covariance).
Note, though, that in practice it might be more robust to wait until we have gotten 
a few observations and use their mean as the mean of the distribution for initializing our particles.

Please implement a function that takes a set of measurements and updates the 
particles to include potentially new landmarks that are not yet tracked by the 
particles. 

Below we give a description for this procedure:

- Filter the input measurements to contain only measurements to the new (untracked) 
landmarks. 
- For each new landmark measurement and each particle:
    - Find the predicted location using the robot's pose of that particle and the landmark's measurement.
    - Add a Gaussian noise of covariance `self.new_landmarks_init_covariance` to this predicted location.
    - This noisy location becomes the landmark's location in the current particle.
- Update `self.particles.landmarks_id_to_idx` to maintain the invariance that it 
holds a mapping from each tracked unique landmark indentifier to the index of 
that landmark in `self.particles.landmarks_loc`. 


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def add_new_landmarks(self: SLAM, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added 
        before).
    """
    new_measurements = [
        m for m in measurements
        if m.landmark_id not in self.particles.landmarks_id_to_idx
    ]

    # Insert new landmarks into the mapping `landmarks_id_to_idx`
    new_landmarks_id_to_idx = dict(self.particles.landmarks_id_to_idx)
    for m in new_measurements:
        new_landmarks_id_to_idx[m.landmark_id] = len(
            new_landmarks_id_to_idx)

    # Find estimated landmark location
    # based on the new measurements and the current pose in the particles
    est_landmarks_loc = ...  # shape: (num_particles, |new_measurements|, 2)

    # Sample new landmark locations around est_landmarks_loc with
    # zero-mean Gaussian noise of `new_landmarks_init_covariance``
    sampled_landmark_locs = ...  # shape: (num_particles, |new_measurements|, 2)

    # Update particles
    new_particles = dataclasses.replace(
        self.particles,
        landmarks_id_to_idx=new_landmarks_id_to_idx,
        landmarks_loc=np.concatenate(
            [self.particles.landmarks_loc, sampled_landmark_locs], axis=1),
    )

    raise NotImplementedError("Implement me!")

    self.particles = new_particles

### Tests

In [ ]:
def slam_add_new_landmarks_test(add_new_landmarks, infer: SLAM,
                                measurements: Sequence[Measurement],
                                results: SLAMParticles):
    add_new_landmarks(infer, measurements)
    assert (np.allclose(infer.particles.x, results.x) and np.allclose(
        infer.particles.y,
        results.y)), "pose component of the particles should not change"
    assert infer.particles.landmarks_id_to_idx == results.landmarks_id_to_idx, "indexing mapping must be the same"
    assert np.allclose(
        infer.particles.landmarks_loc, results.landmarks_loc
    ), "landmark locations should match exactly, since we applied zero initialization noise"

slam_add_new_landmarks_test(add_new_landmarks, SLAM(motion_noise_covariance=np.zeros((2, 2), dtype=np.float64), sensor_noise_covariance=np.zeros((2, 2), dtype=np.float64), new_landmarks_init_covariance=np.zeros((2, 2), dtype=np.float64), num_particles=3, particles=SLAMParticles(x=np.array([0, 1, 2], dtype=np.int64), y=np.array([0, 1, 2], dtype=np.int64), theta=np.zeros((3,), dtype=np.float64), landmarks_id_to_idx={'Stata': 0, 'Building-4': 1}, landmarks_loc=np.array([[[ 2,  3],   [ 1,  1]],   [[ 1,  2],   [ 0,  0]],   [[ 0,  1],   [-1, -1]]], dtype=np.int64))), [Measurement(landmark_id='Stata', r=100, b=0.1), Measurement(landmark_id='Cafe-Luna', r=1, b=1.5707963267948966)], SLAMParticles(x=np.array([0, 1, 2], dtype=np.int64), y=np.array([0, 1, 2], dtype=np.int64), theta=np.zeros((3,), dtype=np.float64), landmarks_id_to_idx={'Stata': 0, 'Building-4': 1, 'Cafe-Luna': 2}, landmarks_loc=np.array([[[ 2.000000e+00,  3.000000e+00],   [ 1.000000e+00,  1.000000e+00],   [ 6.123234e-17,  1.000000e+00]],   [[ 1.000000e+00,  2.000000e+00],   [ 0.000000e+00,  0.000000e+00],   [ 1.000000e+00,  2.000000e+00]],   [[ 0.000000e+00,  1.000000e+00],   [-1.000000e+00, -1.000000e+00],   [ 2.000000e+00,  3.000000e+00]]], dtype=np.float64)))
print('Tests passed.')

## SLAM: Computing the Importance Weights


### Question

Please implement the `SLAM.compute_weights` function that takes a 
sequence of measurements and returns the importance weights of the particles.

This function is similar to `Localization.compute_weights`, except that it takes 
into account the landmarks location tracked in the particles.


For reference, our solution is **30** line(s) of code.

In [ ]:
@implementation_for(SLAM)
def compute_weights(self: SLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def slam_compute_weights_test(_, infer: SLAM,
                              measurements: Sequence[Measurement],
                              result_weights: np.ndarray):
    weights = infer.compute_weights(measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

slam_compute_weights_test(compute_weights, SLAM(motion_noise_covariance=np.array([[0.001     , 0.        ],  [0.        , 0.00274156]], dtype=np.float64), sensor_noise_covariance=np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), new_landmarks_init_covariance=np.array([[10.,  0.],  [ 0., 10.]], dtype=np.float64), num_particles=20, particles=SLAMParticles(x=np.zeros((20,), dtype=np.float64), y=np.zeros((20,), dtype=np.float64), theta=np.zeros((20,), dtype=np.float64), landmarks_id_to_idx={'landmark-17': 0, 'landmark-24': 1, 'landmark-9': 2, 'landmark-16': 3, 'landmark-33': 4, 'landmark-4': 5, 'landmark-1': 6, 'landmark-34': 7, 'landmark-26': 8, 'landmark-20': 9, 'landmark-2': 10, 'landmark-31': 11, 'landmark-19': 12, 'landmark-21': 13, 'landmark-28': 14, 'landmark-27': 15, 'landmark-30': 16, 'landmark-14': 17, 'landmark-13': 18, 'landmark-25': 19, 'landmark-32': 20, 'landmark-7': 21, 'landmark-5': 22, 'landmark-12': 23, 'landmark-11': 24, 'landmark-10': 25, 'landmark-29': 26, 'landmark-8': 27, 'landmark-3': 28, 'landmark-22': 29, 'landmark-0': 30, 'landmark-6': 31, 'landmark-23': 32, 'landmark-18': 33, 'landmark-15': 34, 'landmark-35': 35}, landmarks_loc=np.array([[[ 2.92582568e+00,  2.16864042e+01],   [ 1.14022409e+01, -4.59991183e+00],   [ 2.19765309e+00,  8.50585103e+00],   [ 2.84512281e+00,  1.11639161e+01],   [ 1.70773609e+01,  9.23047418e+00],   [-2.32899042e+00,  8.93408872e+00],   [ 3.81750072e+00, -4.54250241e+00],   [ 1.06863265e+01,  1.33089427e+01],   [ 1.22991120e+01,  1.77182852e+00],   [ 4.47945276e+00,  1.22660540e+01],   [-3.97612185e+00,  5.10560066e+00],   [ 1.63328321e+01,  1.84530550e+00],   [ 1.06110992e+01,  2.38917241e+00],   [ 6.86409326e+00,  5.92729795e+00],   [ 5.94423118e+00,  9.43099878e+00],   [ 1.21512233e+01,  1.05396767e+01],   [ 1.79091114e+01, -4.28674005e+00],   [ 2.65520331e-02, -4.50425512e+00],   [ 6.98805996e+00, -1.90115314e+00],   [ 1.12573904e+01,  2.07745118e+00],   [ 9.71827213e+00,  4.50311350e+00],   [ 7.74152841e+00,  1.68986760e+00],   [-2.48127772e+00,  1.13574959e+01],   [ 1.29088684e+01, -8.60038533e+00],   [ 4.04041145e+00,  1.09961375e+01],   [ 5.74477492e+00,  1.74054813e+01],   [ 8.45472848e+00,  1.79934406e+01],   [ 1.63780436e+00,  1.36582604e-01],   [-9.27599216e+00,  3.94754605e+00],   [ 9.67743597e+00,  1.03034447e+01],   [-5.83283229e+00, -2.38807230e-01],   [ 1.35835028e+00, -8.12297715e+00],   [ 4.71415654e+00,  1.77609684e+01],   [ 8.19516394e+00, -3.28064072e+00],   [ 1.08261317e+01,  9.17852836e+00],   [ 1.23252946e+01,  1.74725445e+01]],   [[ 1.20355363e+00,  2.11610348e+01],   [ 1.08696325e+01, -3.43148440e+00],   [ 7.37637074e-01,  7.96420892e+00],   [-2.88895463e+00,  8.98217369e+00],   [ 1.46934326e+01,  6.95469487e+00],   [-4.27971867e+00,  8.36815501e+00],   [-5.93875603e+00, -1.23785792e+00],   [ 1.42097666e+01,  1.18931254e+01],   [ 1.05530322e+01,  5.41542446e+00],   [ 6.07299074e+00,  4.11763125e+00],   [-5.44701499e+00,  9.42246647e-01],   [ 1.73043899e+01, -2.10249444e+00],   [ 7.25359525e+00, -1.58180299e-01],   [ 3.94797597e+00,  5.82201632e+00],   [ 9.20048050e+00,  1.06710866e+01],   [ 1.11043803e+01,  4.70547362e+00],   [ 1.53188413e+01,  9.28997701e-01],   [ 3.93947331e+00,  7.90016265e+00],   [ 4.44166898e+00,  2.41026332e+00],   [ 1.02265069e+01,  1.99312335e+00],   [ 1.45865131e+01,  3.67192918e+00],   [-1.25926336e+00, -4.62093917e+00],   [ 1.11388565e+00,  1.32867749e+01],   [ 1.00711010e+01, -5.57945600e+00],   [-4.76595647e+00,  1.96531103e+01],   [-6.95384989e-01,  1.40863694e+01],   [ 1.00092958e+01,  1.81763598e+01],   [-3.15515438e+00,  5.33898960e+00],   [-3.15037480e+00,  5.20634333e+00],   [ 1.63243143e+01,  1.22806228e+01],   [ 3.19473711e+00,  1.76449506e+00],   [-8.45449522e-01, -7.03771200e+00],   [ 7.23683931e+00,  1.03192111e+01],   [ 7.69472796e+00, -2.81401135e+00],   [ 1.06669381e+01,  5.14630772e+00],   [ 1.78413302e+01,  1.30034413e+01]],   [[ 2.36197476e+00,  1.46272545e+01],   [ 1.38212703e+01, -2.64079198e+00],   [ 2.80769303e+00,  9.89821522e+00],   [ 4.07348639e+00,  1.47906010e+01],   [ 2.11472649e+01,  4.32887227e+00],   [-1.39411649e+00,  5.67706794e+00],   [-3.73910843e-01, -1.49619070e+00],   [ 1.56993487e+01,  1.35604356e+01],   [ 1.21055912e+01,  6.30870429e+00],   [ 1.26138920e+01,  6.15746363e+00],   [-6.43238697e-01,  2.38363174e+00],   [ 1.53016293e+01, -1.18234570e+00],   [ 1.22013553e+01,  1.38251616e+00],   [ 6.44663447e+00,  7.33865668e+00],   [ 1.47435572e+01,  1.17743617e+01],   [ 8.29310028e+00,  7.65216420e+00],   [ 9.75168373e+00, -7.78310024e-01],   [ 2.68870070e+00,  2.79443480e+00],   [ 6.11515023e+00, -4.35487155e+00],   [ 1.88328278e+01,  1.42719085e-01],   [ 1.25083785e+01,  8.83861806e+00],   [ 3.58383950e+00,  7.84316276e+00],   [-4.28521527e+00,  1.59810236e+01],   [ 1.13596833e+00, -5.15131239e+00],   [-4.31987925e+00,  1.62654060e+01],   [-4.21498679e+00,  1.75400466e+01],   [ 6.58204548e+00,  7.79578082e+00],   [-3.85160922e+00,  8.00518390e-01],   [-8.32868904e+00,  1.15876749e+01],   [ 7.53073201e+00,  1.06528802e+01],   [-2.11812750e+00, -2.39154509e+00],   [ 2.84619999e+00, -9.24247033e+00],   [ 6.78128501e+00,  1.82122425e+01],   [ 1.88018686e+00, -7.94306920e+00],   [ 7.42571319e+00,  6.07134619e+00],   [ 1.80122543e+01,  1.95233420e+01]],   [[ 4.27379130e-01,  1.73604037e+01],   [ 1.64456027e+01, -3.62070354e+00],   [-3.32651412e+00,  1.02414899e+01],   [ 5.31390976e+00,  1.52720447e+01],   [ 1.83878873e+01, -3.52523463e+00],   [-1.05825337e+00,  1.71127359e+01],   [-2.93891836e-01,  6.41441137e-01],   [ 1.78935504e+01,  1.13409569e+01],   [ 1.36460551e+01,  3.51414721e+00],   [ 5.52890479e+00,  1.75109319e+00],   [-7.50295510e+00, -9.25633557e-01],   [ 1.19510945e+01,  6.03993453e+00],   [ 1.12878304e+01, -1.90346583e+00],   [ 6.81612058e+00,  7.20548905e+00],   [ 9.37681331e+00,  1.21120450e+01],   [ 1.37528895e+01,  7.12199415e+00],   [ 1.70773571e+01, -4.06043868e-01],   [ 3.16049934e+00,  5.17911261e+00],   [ 2.36761949e+00, -1.28063689e-01],   [ 1.16486740e+01, -1.10555580e-02],   [ 1.47123684e+01,  7.40079547e+00],   [ 1.80540355e+00,  1.66880689e+00],   [-5.95086849e+00,  1.25552379e+01],   [ 7.13278356e+00, -1.92584082e+00],   [-1.88085066e+00,  2.19473953e+01],   [ 4.17101482e+00,  2.07654537e+01],   [ 1.04267267e+01,  1.34984869e+01],   [-2.43399926e+00,  9.21765638e-01],   [-3.18813650e+00,  1.29012215e+00],   [ 1.21362422e+01,  9.81588160e+00],   [-4.82878549e+00, -6.70917890e+00],   [-6.09514424e-01, -5.69212642e+00],   [ 8.12706555e+00,  1.71784690e+01],   [ 8.03057158e+00, -6.42717220e+00],   [ 7.57540244e+00,  7.90204087e+00],   [ 1.56755635e+01,  2.03134274e+01]],   [[ 9.82448950e-01,  1.33699333e+01],   [ 1.66211781e+01, -8.77622097e+00],   [-4.37622355e+00,  1.00469209e+01],   [ 4.05777612e+00,  1.38889319e+01],   [ 1.58340372e+01,  6.04343938e+00],   [-4.13530662e+00,  1.56812073e+01],   [-2.93371886e+00, -4.78329392e-01],   [ 2.13657633e+01,  1.03583874e+01],   [ 1.86431973e+01,  3.17752054e+00],   [ 1.22775843e+01, -3.07617005e-01],   [-3.86052925e+00, -1.59423712e+00],   [ 1.79979902e+01, -3.36510279e+00],   [ 9.80052867e+00, -8.19337392e-01],   [ 4.38809061e+00,  1.72868499e+00],   [ 9.83377687e+00,  1.74028916e+01],   [ 1.03658081e+01,  1.01355318e+01],   [ 1.58343967e+01, -4.36922768e-01],   [-1.75614552e+00,  3.00353229e+00],   [ 6.34824122e+00, -5.54104959e-01],   [ 1.24021656e+01,  3.00137678e-01],   [ 1.77977000e+01,  6.55004487e+00],   [ 1.48195893e+00,  2.12521080e+00],   [-1.75719275e+00,  1.70340421e+01],   [ 1.07061193e+01, -4.29175870e+00],   [-1.15914369e+00,  1.56513374e+01],   [ 1.77532522e+00,  1.73096873e+01],   [ 9.02757988e+00,  1.30723889e+01],   [ 3.05854953e+00,  2.23958684e+00],   [-4.38988796e+00,  7.97187337e+00],   [ 8.21682262e+00,  1.32622180e+01],   [-6.70189942e+00, -6.49550670e+00],   [ 2.03913268e-02, -1.08140026e+01],   [ 4.17071274e+00,  2.20092399e+01],   [ 1.22904770e+01, -3.62615539e+00],   [ 9.75635832e+00,  4.29936329e+00],   [ 1.40539976e+01,  1.89360226e+01]],   [[ 7.03263264e+00,  1.76964266e+01],   [ 4.60939773e+00, -7.14827651e+00],   [ 1.27312300e+00,  6.33499727e+00],   [ 1.13374387e+01,  1.64861933e+01],   [ 1.78886996e+01,  2.41245809e+00],   [-3.95564063e+00,  1.21576750e+01],   [ 1.89170105e+00,  2.81835972e+00],   [ 1.89578166e+01,  7.34972775e+00],   [ 1.60505575e+01, -8.55809831e-01],   [ 6.25872778e+00,  6.28925840e+00],   [-4.82039453e+00,  2.34940348e-01],   [ 1.27296225e+01, -4.42568805e+00],   [ 1.05854221e+01,  4.05931302e+00],   [ 7.73317854e+00,  5.58353677e+00],   [ 6.79393480e+00,  1.47863360e+01],   [ 1.82866476e+01,  1.55096827e+00],   [ 1.26519118e+01, -5.35843816e+00],   [ 2.23652559e+00,  4.99745668e+00],   [ 8.45066262e+00,  3.15161719e+00],   [ 1.43439707e+01, -6.27885803e-01],   [ 1.06224679e+01,  3.08959712e+00],   [ 6.23816533e+00, -1.03555688e-01],   [-5.03034361e+00,  1.57806962e+01],   [ 8.42164798e+00, -8.25270797e+00],   [ 2.43571170e+00,  1.81793533e+01],   [ 6.16479459e-01,  1.36891468e+01],   [ 1.17523712e+01,  1.55686794e+01],   [-8.92622536e-01, -3.75579037e-02],   [-2.61779010e+00,  6.43222202e+00],   [ 4.94266311e+00,  1.10971104e+01],   [ 1.54758048e-01, -3.44887524e+00],   [-2.17128653e+00, -4.45676288e+00],   [ 1.46371118e+01,  1.15149411e+01],   [ 1.11037831e+01, -2.94581318e+00],   [ 9.51452904e+00,  1.07235569e+01],   [ 1.44615962e+01,  1.16375779e+01]],   [[ 6.76666007e+00,  1.34333299e+01],   [ 9.65164439e+00, -4.55528670e-01],   [-3.11586057e+00,  1.18967993e+01],   [ 4.36655524e+00,  1.45652997e+01],   [ 1.70955463e+01,  7.63412287e-01],   [-9.46145658e+00,  1.20765597e+01],   [ 1.21533053e-01,  2.93217558e-01],   [ 1.30932977e+01,  1.03312310e+01],   [ 1.13255270e+01,  4.82695827e-01],   [ 1.11598999e+01, -5.44572868e-01],   [-2.48462873e+00,  4.63164555e+00],   [ 1.85205804e+01, -2.70875327e+00],   [ 9.22384810e+00,  6.93887074e+00],   [ 1.01443134e+01,  7.16761659e+00],   [ 1.31548064e+01,  1.21710531e+01],   [ 1.82221172e+01,  1.03455534e+01],   [ 1.57440438e+01, -2.41143421e+00],   [ 2.78524374e+00,  8.10440769e+00],   [-1.10053636e+00, -5.11247628e+00],   [ 1.72884223e+01,  8.47014038e+00],   [ 1.56439534e+01,  5.96573055e+00],   [ 3.42121120e+00, -1.95446914e+00],   [-3.07580367e+00,  1.64848229e+01],   [ 7.79534634e+00, -9.02793672e+00],   [-3.64189790e+00,  2.03566426e+01],   [-8.35796080e-01,  9.14745997e+00],   [ 1.75193382e+01,  1.67776638e+01],   [-1.92418112e+00, -2.64171851e+00],   [-2.38708425e+00,  1.03253968e+01],   [ 7.02185695e+00,  7.08704595e+00],   [-4.12598783e+00, -5.93920508e+00],   [-3.22746302e+00, -6.06038162e+00],   [ 5.18790090e+00,  1.68862420e+01],   [ 1.18177929e+01, -6.24950497e+00],   [ 1.51466574e+00,  1.25272266e+01],   [ 1.20786267e+01,  1.48775671e+01]],   [[ 4.88970091e+00,  1.79698135e+01],   [ 1.18349553e+01, -2.62977245e+00],   [-1.50352834e+00,  4.38950342e+00],   [ 7.81493092e+00,  2.14253020e+01],   [ 2.14870163e+01,  6.39299468e+00],   [ 1.95380914e+00,  1.25197544e+01],   [-3.18397564e+00,  1.50680025e+00],   [ 1.53927226e+01,  7.16250141e+00],   [ 8.74419823e+00,  6.04580579e+00],   [ 2.11419159e+00,  6.83748859e+00],   [-9.49959152e+00, -3.29144066e+00],   [ 9.51264765e+00,  6.68977578e+00],   [ 8.28262426e+00,  3.46844381e-01],   [ 1.25387292e+01,  9.80368663e+00],   [ 7.04950268e+00,  1.46465773e+01],   [ 1.18763216e+01,  6.15454159e+00],   [ 1.60915967e+01,  8.37744703e-01],   [ 1.21938350e+00,  5.54623749e+00],   [ 3.31541848e+00, -5.72098100e+00],   [ 1.01386140e+01,  2.27237267e+00],   [ 1.82655992e+01,  8.77570858e+00],   [ 3.23130003e+00, -2.48643424e+00],   [ 2.07861546e+00,  1.70424329e+01],   [ 2.63156900e+00, -4.67950911e+00],   [ 7.95673197e-01,  1.77086205e+01],   [ 4.10513157e+00,  1.57789312e+01],   [ 1.23283283e+01,  1.39448303e+01],   [ 3.34864032e-01,  2.85285370e+00],   [ 1.18628947e+00,  2.97165263e+00],   [ 7.54027174e+00,  8.72553789e+00],   [-2.98865872e+00, -1.10042514e+01],   [ 3.27739951e+00, -5.69051771e+00],   [ 2.67923199e+00,  1.49643295e+01],   [ 1.18208598e+01, -8.62695410e+00],   [ 5.85162618e+00,  4.24600169e+00],   [ 1.32938433e+01,  1.70490434e+01]],   [[ 4.79620212e+00,  1.45858226e+01],   [ 1.35081584e+01, -1.50025974e+00],   [-1.11750315e+00,  1.19876295e+01],   [ 7.92844819e+00,  1.22004837e+01],   [ 1.41068608e+01,  6.81985663e-01],   [-4.24426991e+00,  1.02456299e+01],   [ 7.88912339e-01, -4.74199063e+00],   [ 1.48140293e+01,  1.33348328e+01],   [ 9.27534981e+00,  1.09869664e+00],   [ 3.37781133e+00,  1.32140226e+01],   [-6.71011343e+00, -6.95749509e-01],   [ 1.27855930e+01, -1.97821403e+00],   [ 8.05128527e+00,  2.68310050e+00],   [ 7.10726998e+00,  3.93132733e+00],   [ 6.86029340e+00,  1.19676570e+01],   [ 1.14284097e+01,  1.20762558e+01],   [ 1.63071275e+01, -3.52246803e+00],   [ 6.98596358e-01,  3.31101867e+00],   [ 1.95937936e+00, -1.20840935e+00],   [ 9.64212934e+00,  3.36372544e+00],   [ 1.37125913e+01,  4.97564854e+00],   [-4.07754147e+00,  1.65273344e+00],   [-2.07724562e+00,  1.37082226e+01],   [ 8.94501494e+00, -8.41685724e+00],   [-2.10767190e+00,  1.68268990e+01],   [-2.86465322e+00,  1.28886368e+01],   [ 1.64457386e+01,  1.68710023e+01],   [-1.55768457e+00,  4.12364122e+00],   [-4.35916393e+00,  8.20232573e+00],   [ 6.62093513e+00,  1.50692854e+01],   [-2.60520807e+00, -4.96779953e+00],   [-5.89839635e-01, -8.07682047e+00],   [ 6.58910375e+00,  1.40801169e+01],   [ 6.30814360e+00, -1.13638846e+00],   [ 4.92108272e+00,  9.34233225e+00],   [ 1.29328981e+01,  1.67736062e+01]],   [[ 1.96757369e+00,  1.83502317e+01],   [ 1.18599296e+01, -6.39815727e+00],   [-5.42702022e+00,  8.33234889e+00],   [ 1.06441075e+00,  1.35062604e+01],   [ 1.45689026e+01,  8.96197511e+00],   [-2.12328575e+00,  1.28091299e+01],   [-3.62300999e+00,  1.50651712e+00],   [ 2.11139916e+01,  7.63979077e+00],   [ 1.21299259e+01, -1.70578650e+00],   [ 1.23760801e+01,  3.66873193e+00],   [-1.87166398e+00, -2.03301797e+00],   [ 2.10119098e+01, -6.58204855e-01],   [ 1.06286905e+01,  5.49973369e-01],   [ 7.24278111e+00,  1.21804942e+01],   [ 6.23824901e+00,  1.28431535e+01],   [ 1.41736038e+01,  7.14060926e+00],   [ 1.56410932e+01, -3.66059442e+00],   [ 1.38928287e-01,  4.39909504e+00],   [ 4.96731251e+00,  1.79082324e-01],   [ 1.18145486e+01,  1.12731267e-01],   [ 1.32798296e+01,  9.58655831e+00],   [-5.25577091e+00,  2.18410846e-01],   [-4.65377536e+00,  2.09654518e+01],   [ 5.98945803e+00,  5.94634271e-01],   [-8.75484313e+00,  1.91378670e+01],   [ 2.18623990e+00,  1.66859990e+01],   [ 5.87133899e+00,  1.61668579e+01],   [ 1.15419667e-01,  1.14218370e+00],   [-8.46323383e-01,  7.04352705e+00],   [ 1.44011019e+01,  1.39062071e+01],   [-1.04155558e+00, -1.52733744e+00],   [-3.61200232e+00, -7.09724056e+00],   [ 1.03238881e+01,  1.82488930e+01],   [ 1.40288301e+01, -5.07253230e+00],   [ 3.12139281e+00,  1.03067530e+01],   [ 1.24917583e+01,  1.97261437e+01]],   [[ 8.00998039e+00,  1.75169237e+01],   [ 1.15147363e+01, -3.95701865e+00],   [ 1.64120525e+00,  7.49282416e+00],   [ 1.02561444e+01,  1.04233822e+01],   [ 1.29531733e+01,  2.85534826e+00],   [-5.70843796e+00,  4.50453461e+00],   [-3.66579341e+00, -2.03388908e+00],   [ 1.59395011e+01,  1.15925955e+01],   [ 7.66303332e+00,  6.35571692e+00],   [ 1.36526288e+01,  4.26838514e+00],   [ 6.32203298e-01,  4.04556826e+00],   [ 2.16165137e+01,  4.00348255e+00],   [ 3.95766986e+00,  1.31949242e-01],   [ 1.13847190e+01,  6.94614766e+00],   [ 1.13331927e+01,  1.59138867e+01],   [ 9.04913329e+00,  7.29041183e+00],   [ 1.70381766e+01, -4.26491596e+00],   [ 3.95740170e+00,  6.72936441e+00],   [ 4.71471008e+00, -2.59523687e+00],   [ 9.18401463e+00, -8.33982431e-01],   [ 1.41669912e+01,  5.10344785e+00],   [-1.28813233e+00, -1.79982404e+00],   [-8.42410958e+00,  1.76831522e+01],   [ 5.68742588e+00, -8.08491015e+00],   [-1.94800720e+00,  2.19302755e+01],   [-1.03926343e+00,  1.64475749e+01],   [ 1.00388767e+01,  1.72827117e+01],   [ 2.15633956e-01,  7.14203291e+00],   [-4.19246809e+00,  4.50352482e+00],   [ 3.82096545e+00,  5.72508186e+00],   [-9.46208074e+00, -3.15010425e+00],   [-3.27894772e+00, -5.98954300e+00],   [ 6.28689156e+00,  1.36688355e+01],   [ 8.37458533e+00, -1.36942116e+00],   [ 8.77965159e+00,  8.76377766e+00],   [ 1.19219286e+01,  1.89998300e+01]],   [[ 6.39456402e+00,  1.69429518e+01],   [ 1.07366850e+01, -1.84532126e+00],   [ 1.10157096e+00, -2.39575125e+00],   [ 5.34245725e+00,  7.50437301e+00],   [ 1.82222557e+01,  5.45734301e+00],   [-6.94909893e-02,  1.10845081e+01],   [-4.47838440e+00, -4.33994665e-01],   [ 1.90426197e+01,  1.03543932e+01],   [ 1.12233822e+01,  5.86081817e+00],   [ 8.61865315e+00,  4.90829636e+00],   [-2.65344582e-02,  8.33401059e+00],   [ 1.72670990e+01, -4.33960231e-01],   [ 1.36368187e+01,  3.81439687e+00],   [ 1.03240031e+01,  3.00397908e+00],   [ 1.18946723e+01,  9.54684006e+00],   [ 1.07910484e+01,  7.99414927e+00],   [ 1.63315139e+01, -4.71201188e+00],   [ 8.04648872e+00, -1.08511235e+00],   [ 5.65500041e+00, -5.91172533e+00],   [ 1.38854428e+01, -1.14325997e+00],   [ 1.69404836e+01,  5.88260876e+00],   [ 2.12606461e+00, -3.12838819e+00],   [-5.68725582e+00,  1.70170617e+01],   [ 7.65938635e+00,  5.16150283e+00],   [-1.33377328e+00,  2.07710335e+01],   [-3.62283821e+00,  8.08325713e+00],   [ 9.93756584e+00,  1.28705253e+01],   [ 4.60692593e+00,  1.41280131e+00],   [-3.22637001e-01,  1.28161532e+01],   [ 8.10189911e+00,  1.21594340e+01],   [-4.22575785e+00, -1.59330882e+00],   [ 1.28015963e+00, -6.12500869e+00],   [ 1.01095446e+01,  2.15570757e+01],   [ 1.21499071e+01, -2.56408520e+00],   [ 6.23599251e+00,  4.38767339e+00],   [ 1.18034243e+01,  1.79870324e+01]],   [[-1.14313479e-01,  1.36511005e+01],   [ 1.66979951e+01,  1.96639375e+00],   [-5.50371446e+00,  1.26045323e+01],   [ 2.63589440e+00,  1.35087085e+01],   [ 1.57652507e+01,  9.36154327e+00],   [-8.92794227e+00,  1.28374371e+01],   [-4.66260585e+00,  1.83697035e+00],   [ 2.05699683e+01,  1.02700744e+01],   [ 1.27681240e+01,  4.80461194e+00],   [ 5.42490177e+00,  9.70726381e+00],   [ 1.04993072e+00,  5.63554211e+00],   [ 1.55593401e+01, -9.90314488e-01],   [ 6.92203451e+00,  5.48318463e+00],   [ 1.25234238e+01,  6.64823558e+00],   [ 6.45364969e+00,  2.94293010e+00],   [ 7.86779339e+00,  6.82196464e+00],   [ 2.04441497e+01, -5.27009413e+00],   [ 5.61742215e+00,  8.39954570e+00],   [ 3.75242487e+00, -4.88307194e-01],   [ 8.53688762e+00, -2.14118304e+00],   [ 1.54788658e+01,  4.97389647e+00],   [ 1.73022632e+00, -5.78114896e-02],   [-1.51711541e+00,  1.38803833e+01],   [ 4.75277703e+00,  1.89514192e+00],   [-4.56765901e+00,  1.67941122e+01],   [-5.63185213e-01,  1.36952666e+01],   [ 1.42948274e+01,  1.50816407e+01],   [-1.45773316e+00,  1.82780942e+00],   [-3.01799837e+00,  5.77636092e+00],   [ 6.77772796e+00,  3.07106818e+00],   [-8.41781711e+00, -3.65157449e+00],   [ 2.59099430e+00, -4.80088269e+00],   [ 5.03807832e+00,  1.16782706e+01],   [ 1.04475388e+01, -8.82097807e+00],   [ 9.21219321e+00,  7.79208449e+00],   [ 9.96922967e+00,  1.39098749e+01]],   [[ 1.92560798e+00,  1.36293917e+01],   [ 7.93391858e+00, -6.85285781e+00],   [-2.39976652e+00,  5.21133167e+00],   [ 6.87143302e+00,  1.38876533e+01],   [ 1.29829346e+01,  4.88639629e+00],   [-6.62542630e+00,  1.04462167e+01],   [ 1.09393911e+00,  8.29728791e-01],   [ 2.29189235e+01,  1.14601810e+01],   [ 8.69590223e+00,  6.07012052e+00],   [ 1.13684808e+01,  8.38710030e+00],   [-7.80668942e+00,  2.94943773e+00],   [ 1.89066949e+01, -4.85725874e+00],   [ 7.32387608e+00, -4.43932884e+00],   [ 5.10160467e+00,  3.64788025e+00],   [ 1.54044778e+01,  1.11568636e+01],   [ 8.56371924e+00,  3.96014973e+00],   [ 1.64627730e+01, -3.24459020e+00],   [ 8.27139188e+00,  5.85971729e+00],   [-1.18178478e+00, -2.02685912e+00],   [ 1.60594276e+01, -3.02658175e-01],   [ 1.96482421e+01,  2.13612574e+00],   [ 3.15351068e+00, -2.63170893e+00],   [-5.22880461e+00,  1.98358030e+01],   [ 5.41215455e+00, -1.52875091e+00],   [-5.41836901e-01,  2.00684645e+01],   [-1.50807855e+00,  1.15880647e+01],   [ 1.24033436e+01,  1.39721200e+01],   [ 5.05945036e+00,  2.52215843e-01],   [-5.64217452e+00,  4.93151492e+00],   [ 1.18888255e+00,  1.62549592e+01],   [-4.69453219e+00, -4.71242720e+00],   [-8.76693715e-01, -1.24399012e+00],   [ 5.46075420e+00,  1.82552518e+01],   [ 8.55501570e+00, -5.48119295e+00],   [ 7.63463216e+00,  1.07151181e+01],   [ 1.34269185e+01,  1.45331747e+01]],   [[ 5.19076116e-01,  1.95640264e+01],   [ 1.08263813e+01, -9.93439685e+00],   [ 4.22316138e+00,  3.12873862e+00],   [ 5.76939160e+00,  1.41362110e+01],   [ 1.70343583e+01,  4.09660584e+00],   [-2.68792083e+00,  6.12093799e+00],   [ 1.84874706e-01,  4.95567661e+00],   [ 1.84867574e+01,  3.88504758e+00],   [ 8.47259959e+00,  2.91563581e+00],   [ 9.49326939e+00,  4.58977510e+00],   [-5.90607653e+00,  5.29881309e+00],   [ 1.90089623e+01, -2.64256144e-01],   [ 1.12082492e+01, -8.08263023e+00],   [ 5.08598569e+00,  5.83364092e+00],   [ 9.90484666e+00,  1.29509383e+01],   [ 1.44996500e+01,  8.83725074e+00],   [ 1.40471323e+01, -8.02479177e+00],   [ 5.78233071e+00, -1.53429958e+00],   [ 5.41104256e+00, -6.33775803e-01],   [ 8.95724453e+00,  2.56976515e+00],   [ 1.41539482e+01,  5.49349107e+00],   [ 1.07641082e+00,  1.41834549e+00],   [-6.94006249e-01,  1.79322180e+01],   [ 9.01462238e+00, -4.24069482e+00],   [ 3.07950459e+00,  1.94905397e+01],   [-4.18692296e-01,  8.82954456e+00],   [ 1.17125257e+01,  1.25255403e+01],   [-3.17156630e+00,  3.46608645e+00],   [-6.61578880e+00,  5.16312907e+00],   [ 7.85759809e+00,  1.08024316e+01],   [-4.69926008e+00, -4.48990430e+00],   [ 2.45431609e+00, -4.65114388e+00],   [ 7.80615386e+00,  2.38991397e+01],   [ 1.05081016e+01, -1.46394925e-02],   [ 4.06745895e+00,  3.28002928e+00],   [ 1.25482684e+01,  1.61169679e+01]],   [[ 2.64566843e+00,  2.03004653e+01],   [ 1.22658971e+01, -2.62763934e+00],   [-3.82485093e-01,  6.08959554e+00],   [ 3.42467492e+00,  1.89764654e+01],   [ 1.36954231e+01,  8.36999948e+00],   [-7.08854235e+00,  9.94145465e+00],   [-1.66333284e+00,  1.06406719e+00],   [ 2.03812824e+01,  1.01130189e+01],   [ 8.91719495e+00,  5.56510871e+00],   [ 6.80183238e+00,  5.68181787e+00],   [-4.71392332e+00, -2.72944196e+00],   [ 1.69349743e+01,  2.31509362e+00],   [ 1.18333521e+01,  5.76660750e+00],   [ 4.27140071e+00,  8.48289084e+00],   [ 1.27441331e+01,  1.05732843e+01],   [ 1.25496495e+01,  8.81504501e+00],   [ 1.21609654e+01, -2.43075152e+00],   [ 1.11813425e+00,  2.21717441e+00],   [-2.26627854e+00, -1.53474616e+00],   [ 1.20318143e+01,  3.40017785e+00],   [ 1.41658891e+01,  7.32078508e-01],   [ 4.06909981e+00, -2.83310056e-01],   [-2.07295990e+00,  1.94163410e+01],   [ 1.03126474e+01,  2.10083152e+00],   [-2.26695219e+00,  1.86683938e+01],   [-2.10945800e+00,  1.56458190e+01],   [ 7.31479593e+00,  1.09484170e+01],   [ 2.53242989e+00,  1.06027745e+00],   [-2.23446977e+00,  9.40497951e+00],   [ 2.81209679e+00,  1.97222257e+00],   [-7.51611494e+00, -3.24384545e+00],   [-4.11189752e+00, -7.79820719e+00],   [ 3.89911359e+00,  1.66434708e+01],   [ 1.01823686e+01, -5.11665118e+00],   [ 1.02593450e+01,  1.23262178e+01],   [ 1.68395541e+01,  1.12731130e+01]],   [[ 7.22744506e-01,  1.73619003e+01],   [ 1.06578268e+01, -5.62725041e+00],   [ 2.60728574e+00,  9.07614276e+00],   [ 7.46024219e+00,  1.21988797e+01],   [ 2.04727344e+01,  4.19224585e+00],   [ 8.97785899e-01,  1.17295996e+01],   [-4.39583468e+00, -5.12639047e+00],   [ 1.47378788e+01,  9.46111514e+00],   [ 1.42971068e+01,  3.15977762e+00],   [ 6.58596844e+00,  8.33293201e+00],   [-5.05735971e+00,  1.59407161e+00],   [ 1.71446641e+01, -1.07865083e+00],   [ 1.09864858e+01, -1.81457332e+00],   [ 7.34559183e+00,  7.33077673e+00],   [ 1.02667374e+01,  1.18560007e+01],   [ 1.38726381e+01,  8.53345947e+00],   [ 1.75357423e+01, -5.21878374e+00],   [ 6.71794004e+00,  6.39586769e+00],   [ 6.52220972e+00, -2.33481793e+00],   [ 1.21301993e+01,  3.66478787e+00],   [ 1.22378008e+01,  4.89415984e+00],   [ 7.43268073e+00,  6.26370402e+00],   [ 3.03712996e-01,  1.93298496e+01],   [ 2.59082106e+00, -2.48024868e+00],   [-3.17413956e+00,  1.43370336e+01],   [-3.08998604e-01,  1.26545095e+01],   [ 9.33084872e+00,  1.49153434e+01],   [ 2.41117391e+00,  3.16573144e+00],   [-3.95522064e+00,  8.46830574e+00],   [ 5.82985681e+00,  6.23847490e+00],   [-3.06626389e+00, -4.89938673e+00],   [-6.60695336e+00, -6.97540091e+00],   [ 5.05879287e+00,  1.83155955e+01],   [ 1.15446756e+01, -2.02303995e+00],   [ 3.73839039e+00,  9.90339621e+00],   [ 1.60628437e+01,  1.56684154e+01]],   [[ 2.78250265e+00,  1.88974322e+01],   [ 1.11478329e+01, -3.12668597e+00],   [ 3.03416031e+00,  1.34984863e+01],   [ 8.56614979e+00,  1.64048104e+01],   [ 1.13617799e+01,  3.76560026e+00],   [-5.57722641e+00,  1.66226399e+01],   [-3.88610143e+00,  1.25314384e+00],   [ 1.60101876e+01,  7.31426220e+00],   [ 1.27923753e+01,  4.76901831e+00],   [ 3.98232957e+00,  8.79411850e+00],   [-1.09445197e+01,  1.54817008e+00],   [ 1.84012311e+01, -3.30309058e+00],   [ 1.00108086e+01, -4.02640717e+00],   [ 7.75648250e+00, -4.67242142e-01],   [ 8.78538304e+00,  7.01939194e+00],   [ 1.00234008e+01,  5.92194563e+00],   [ 1.38844689e+01, -6.86153183e-01],   [ 1.21649596e+00,  6.28463247e+00],   [ 8.49817971e+00,  4.01911975e-01],   [ 9.96336991e+00, -2.28937751e+00],   [ 1.19185057e+01,  7.83749781e+00],   [ 1.83476937e+00,  3.05410036e+00],   [-1.18132317e+00,  1.68070652e+01],   [ 7.31127756e+00,  1.76569086e+00],   [ 2.43818488e+00,  1.67271649e+01],   [ 4.43026038e+00,  1.41413674e+01],   [ 1.46553137e+01,  1.87809622e+01],   [-3.17406914e+00, -2.41186560e+00],   [-2.88267428e+00,  1.41311412e+01],   [ 5.51375248e+00,  9.97781189e+00],   [-7.32367763e+00,  4.28627816e-01],   [-7.53787916e+00, -1.28923166e+01],   [ 9.69860638e+00,  1.58458760e+01],   [ 8.37845067e+00, -4.54649935e+00],   [ 7.53954153e+00,  6.28146388e+00],   [ 1.15832410e+01,  1.40319101e+01]],   [[ 7.02525562e+00,  1.44150613e+01],   [ 8.26486809e+00, -3.76381582e+00],   [ 1.36143476e+00,  1.16631576e+01],   [ 3.99359199e+00,  1.14699686e+01],   [ 1.76228342e+01,  8.24409156e+00],   [-3.96705192e+00,  1.12538854e+01],   [-8.52051456e-01, -1.88927107e-01],   [ 1.64821475e+01,  7.03982040e+00],   [ 7.94693808e+00,  8.65479162e-01],   [ 9.98428775e+00,  5.74543023e+00],   [-1.05687126e+00,  2.82593970e+00],   [ 1.17110796e+01,  4.08971808e+00],   [ 8.84382391e+00,  4.13168551e+00],   [ 9.94152350e+00,  1.37705708e+01],   [ 1.07548455e+01,  1.47468771e+01],   [ 1.41126301e+01,  5.47955896e+00],   [ 1.54693451e+01, -7.83179876e+00],   [-8.76667910e-01,  4.67877135e+00],   [ 5.85366589e+00,  1.97976167e+00],   [ 1.37508134e+01,  4.39047763e+00],   [ 1.22982551e+01,  4.74739765e+00],   [ 3.67499607e+00,  1.46133546e+00],   [-4.28199870e+00,  2.05385014e+01],   [ 1.72830951e+00, -5.74131880e+00],   [-2.93469563e+00,  2.02666419e+01],   [ 1.30816185e+00,  8.29955729e+00],   [ 1.26871630e+01,  2.00962475e+01],   [ 2.24440499e+00,  4.66796649e+00],   [-1.06205796e+01,  5.26884326e+00],   [ 9.49109050e+00,  1.03990322e+01],   [-2.39588351e+00, -7.30052327e+00],   [-4.42765034e-01, -1.01288677e+01],   [ 2.33461519e+00,  1.70416051e+01],   [ 1.25596515e+01, -2.81764670e+00],   [ 5.73709261e+00,  2.65050393e+00],   [ 1.82336122e+01,  1.68252070e+01]],   [[ 1.04127606e+01,  1.90543006e+01],   [ 6.30473710e+00, -1.69925532e+00],   [-4.65173096e+00,  5.05887910e+00],   [ 2.51155874e+00,  1.32587071e+01],   [ 1.73811013e+01,  2.71001107e+00],   [-7.48283016e+00,  1.32942015e+01],   [-1.71656496e+00,  5.83394533e-01],   [ 8.75418206e+00,  9.71123649e+00],   [ 8.76820874e+00,  5.98219286e-01],   [ 1.08688523e+01,  1.46222382e+00],   [-7.80243822e+00,  2.36956079e+00],   [ 1.67515295e+01, -2.32517803e+00],   [ 1.28058767e+01,  4.65786476e+00],   [ 5.26327031e+00,  6.07453432e+00],   [ 8.98744066e+00,  6.56889179e+00],   [ 1.22520944e+01,  9.55485326e+00],   [ 2.23768555e+01, -6.04723428e+00],   [ 8.24660189e+00,  8.52342075e+00],   [ 3.97162444e+00,  1.86636067e+00],   [ 1.54370931e+01,  7.90402993e+00],   [ 1.56740066e+01,  5.41904918e+00],   [-1.39631798e+00,  1.05720278e+00],   [-3.78629418e-01,  1.71353087e+01],   [ 3.34764234e+00, -3.81942909e+00],   [-2.44693235e+00,  1.85217093e+01],   [-3.95460678e+00,  1.40304910e+01],   [ 1.29446324e+01,  1.80638742e+01],   [-1.05380296e-01, -3.00084518e+00],   [-4.75458394e+00,  1.14688095e+01],   [ 5.24665143e+00,  4.28198147e+00],   [-8.92002780e+00, -2.30295968e+00],   [ 9.30835958e-01, -2.93128907e+00],   [ 8.33794498e+00,  1.13618146e+01],   [ 7.30681445e+00, -1.00495658e+00],   [ 6.28572371e+00,  9.64496872e+00],   [ 1.64269452e+01,  1.63163470e+01]]], dtype=np.float64))), [Measurement(landmark_id='landmark-17', r=16.618152723564037, b=1.3142893395563815), Measurement(landmark_id='landmark-24', r=13.2895332911168, b=-0.31259629224640056), Measurement(landmark_id='landmark-9', r=7.464330626838889, b=1.6023514481303975), Measurement(landmark_id='landmark-16', r=13.953110685803654, b=1.3316941778460758), Measurement(landmark_id='landmark-33', r=17.184808584191327, b=0.3532187535248875), Measurement(landmark_id='landmark-4', r=12.025836178136165, b=1.8961532532226417), Measurement(landmark_id='landmark-1', r=1.6749692253611657, b=3.122499479032094), Measurement(landmark_id='landmark-34', r=18.754089052746934, b=0.5795987270710623), Measurement(landmark_id='landmark-26', r=12.104851657816209, b=0.29414815863056887), Measurement(landmark_id='landmark-20', r=9.355902446373292, b=0.5546240629649799), Measurement(landmark_id='landmark-2', r=5.5283195865483465, b=2.475440923538948), Measurement(landmark_id='landmark-31', r=15.334805326513386, b=0.03067504038519622), Measurement(landmark_id='landmark-19', r=8.903470181651809, b=0.00820412066646277), Measurement(landmark_id='landmark-21', r=10.570209249630953, b=0.7049624503229832), Measurement(landmark_id='landmark-28', r=16.512836922809804, b=0.8046138129043872), Measurement(landmark_id='landmark-27', r=13.412586918317222, b=0.569748591066251), Measurement(landmark_id='landmark-30', r=16.333197492556163, b=-0.19778098213865833), Measurement(landmark_id='landmark-14', r=5.871513371998722, b=0.8164102826198404), Measurement(landmark_id='landmark-13', r=3.3461713905816604, b=-0.011310923317063418), Measurement(landmark_id='landmark-25', r=12.78397547006133, b=0.13032645317965927), Measurement(landmark_id='landmark-32', r=15.233356970366522, b=0.37709343559395214), Measurement(landmark_id='landmark-7', r=1.3458754237823045, b=0.06818228212795595), Measurement(landmark_id='landmark-5', r=16.756878132799947, b=1.7883800563678198), Measurement(landmark_id='landmark-12', r=7.1148749330293395, b=-0.6143333543391841), Measurement(landmark_id='landmark-11', r=17.801634869866124, b=1.6855607802948347), Measurement(landmark_id='landmark-10', r=12.357380410658957, b=1.465350634062304), Measurement(landmark_id='landmark-29', r=19.995545866879915, b=0.9845834633711026), Measurement(landmark_id='landmark-8', r=2.7116385362504456, b=1.6052772312820927), Measurement(landmark_id='landmark-3', r=9.374135604821388, b=2.0951851221342133), Measurement(landmark_id='landmark-22', r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id='landmark-0', r=5.22041900234916, b=-2.4582789646260825), Measurement(landmark_id='landmark-6', r=5.7393678771301335, b=-1.6140727018312917), Measurement(landmark_id='landmark-23', r=18.217513449458522, b=1.0845840064251768), Measurement(landmark_id='landmark-18', r=10.527744788801282, b=-0.34842437630350404), Measurement(landmark_id='landmark-15', r=9.577624532824075, b=0.9148562052065338), Measurement(landmark_id='landmark-35', r=22.67944597222941, b=0.8450610387947912)], np.array([0.00000000e+000, 9.88131292e-324, 4.45810892e-175, 3.56795095e-066, 5.79754745e-098, 1.02078810e-216, 0.00000000e+000, 4.15469530e-122, 7.88047520e-274, 2.51973479e-322, 1.37155049e-170, 0.00000000e+000, 1.32313709e-089, 0.00000000e+000, 5.47806515e-123, 1.52801009e-264, 1.00000000e+000, 5.03346026e-218, 1.82949973e-015, 0.00000000e+000], dtype=np.float64))
print('Tests passed.')

## SLAM: Particle Filter Update


### Question

Please implement the update step of SLAM.

The procedure is similar to `Localization.update`, except that it does an additional `add_new_landmarks` 
step after forward updating with the `motion_model` and before `compute_weights`. 


For reference, our solution is **20** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `add_new_landmarks`, `compute_weights`, `motion_model`. You may not need to use all of them.

In [ ]:
@implementation_for(SLAM)
def update(self: SLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the particles, based on the command and measurements.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def slam_update_test(_, infer: SLAM, command: Command,
                     measurements: Sequence[Measurement], pose: Pose,
                     landmarks: Sequence[Landmark]):
    """Checks if estimation after one step update matchs our estimation."""
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    est_landmarks = infer.estimated_landmarks()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=1.0), "Estimated pose should be quite close to our estimation"

    landmarks = {l.id: l for l in landmarks}
    est_landmarks = {l.id: l for l in est_landmarks}

    assert landmarks.keys() == est_landmarks.keys()

    for lid in landmarks:
        assert np.allclose(
            [landmarks[lid].x, landmarks[lid].y],
            [est_landmarks[lid].x, est_landmarks[lid].y],
            atol=1.0
        ), "Estimated landmark location should be quite close to our estimation"

slam_update_test(update, SLAM(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 5.79948578e-09]], dtype=np.float64), new_landmarks_init_covariance=np.zeros((2, 2), dtype=np.float64), num_particles=100, particles=SLAMParticles(x=np.zeros((100,), dtype=np.float64), y=np.zeros((100,), dtype=np.float64), theta=np.zeros((100,), dtype=np.float64), landmarks_id_to_idx={}, landmarks_loc=np.array([], dtype=np.float64).reshape((100, 0, 2)))), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id='L-only', r=1.4143392925941884, b=0.7853881030367988)], Pose(x=1.0006045140443256, y=-0.00012787494016420898, theta=0.7839209541825823), [Landmark(id='L-only', x=1.0025957357392745, y=1.4142827160763445)])
print('Tests passed.')

## Einsum Warmup (Optional)


### Question

This question intends to get you familiar with using `numpy.einsum`.
**This question is optional and you do not have to use `einsum` in your implementation.** 
However, using `einsum` can greatly simplify your implementation.

In particular, the Rao-blackwellized particle filter performs quite a few 
matrix multiplications. 
The function `numpy.einsum` allows one to express batched multiplication of 
multiple matrices succinctly.

If you are not familar with `np.einsum`, here are some guides online:
- <a href="https://stackoverflow.com/questions/26089893/understanding-numpys-einsum" target="_blank">StackOverflow Post</a>
- <a href="https://rockt.github.io/2018/04/30/einsum" target="_blank">Blog: einsum is all you need</a>

For this warmup question, please fill in the indexing expression string for 
`einsum` to implement a "sandwich matrix product". 
A sandwich matrix product of two matrices $U$ and $S$ is
$M = U \cdot S \cdot U^\top$.


For reference, our solution is **2** line(s) of code.

In [ ]:
def sandwich_matrix_product(U: np.ndarray, S: np.ndarray) -> np.ndarray:
    """Perform a batched 'sandwitch matrix product': $$M = U \cdot S \cdot U^\top$$.
    Uses `numpy.einsum`.

    Args:
        U, S: input matrices to perform the product, both of shape (batch, N, N)

    Returns:
        the matrices M, of shape (batch, N, N)
    """
    return np.einsum("FILL IN THE STRING HERE", U, S, U)

### Tests

In [ ]:

assert np.allclose(sandwich_matrix_product(np.array([[[ 5.29927308e-02,  6.97486326e-04,  8.73758417e-01],   [ 7.60854757e-01,  4.27867933e-01,  1.55081421e-01],   [-4.95261373e-01, -9.83674465e-01, -7.60730520e-01]],   [[-3.59411540e-01, -6.06544308e-01,  1.40007645e-01],   [ 6.98996350e-02, -2.20417110e-01,  6.01089051e-01],   [ 4.62314292e-01,  3.75765732e-01,  5.35049489e-03]],   [[-9.13109541e-01, -1.43944542e-01, -1.63633090e-03],   [-1.04304807e-01, -9.91050747e-01,  3.04857718e-01],   [-4.85214449e-01, -9.10682351e-01,  6.93721876e-03]],   [[ 8.41094765e-01, -9.41119433e-01, -9.72516814e-01],   [ 1.75496174e-01,  8.59724746e-02, -9.68587303e-01],   [-7.93862499e-01,  6.72789285e-01, -9.47231070e-02]],   [[-7.51225467e-01, -6.88159032e-01,  2.22845472e-01],   [-3.42668010e-01, -9.48589536e-01,  2.72036992e-01],   [-9.35294148e-01, -7.79440103e-01, -6.82463503e-01]]], dtype=np.float64), np.array([[[-0.71344418,  0.52312589, -0.81310909],   [-0.70271682, -0.10596886, -0.71983262],   [ 0.49387206, -0.0245083 ,  0.75928679]],   [[-0.92376825,  0.71039678, -0.69237809],   [ 0.85126947, -0.34791778,  0.02156608],   [-0.95261185,  0.45827782, -0.57245743]],   [[ 0.83323349,  0.32357409, -0.43276171],   [-0.47818282, -0.65904157,  0.60374101],   [-0.1230465 , -0.44610878,  0.90563902]],   [[ 0.60448166,  0.17260602,  0.60489731],   [-0.75974651,  0.34432755, -0.35169685],   [ 0.37501795,  0.92331426,  0.56071918]],   [[ 0.13111767,  0.49106642, -0.06333667],   [-0.44010964, -0.91873192,  0.35440609],   [-0.71481713, -0.08130408,  0.33350909]]], dtype=np.float64)), np.array([[[ 0.56243495,  0.39798211, -0.67241513],   [-0.74718029, -0.55967505,  0.6519886 ],   [ 0.50121255,  0.53333106, -0.60289277]],   [[ 0.12392122,  0.06695595, -0.13857656],   [-0.00288673, -0.38500122, -0.23246075],   [-0.12514734, -0.24134036,  0.02160777]],   [[ 0.65995668,  0.36375436,  0.5198115 ],   [-0.38824333, -0.59999865, -0.61171386],   [-0.09048873, -0.54099004, -0.4177997 ]],   [[ 1.44931832, -0.22707922, -1.43407209],   [ 0.96840162,  0.32417985, -0.26113564],   [-0.12140057,  0.56621217,  0.89269574]],   [[-0.22979006, -0.28123713, -0.15003431],   [-0.65236516, -0.7679908 , -0.54475157],   [-0.8460349 , -0.6598786 , -0.60239911]]], dtype=np.float64), atol=1e-6)
print('Tests passed.')

## RBSLAM: Update by Motion Model


### Utilities

Rao-blackwellized Particle Filter for SLAM
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:


@dataclasses.dataclass(frozen=True)
class RBSLAMParticles:
    """A batch of particles for Rao-blackwellized SLAM."""
    x: np.ndarray  # shape: (num_particles, )
    y: np.ndarray  # shape: (num_particles, )
    theta: np.ndarray  # shape: (num_particles, )

    # Mapping from landmark_id to the index of that landmark in `landmarks_loc`
    landmarks_id_to_idx: Dict[str, int]

    landmarks_loc: np.ndarray  # shape: (num_particles, num_landmarks, 2)
    landmarks_cov: np.ndarray  # shape: (num_particles, num_landmarks, 2, 2)

    def __post_init__(self):
        assert self.x.shape == self.y.shape == self.theta.shape
        assert (self.landmarks_loc.shape + (2,) == self.landmarks_cov.shape)
        assert self.landmarks_loc.shape[1] == len(self.landmarks_id_to_idx)
        assert self.landmarks_loc.shape[0] == self.x.shape[0]
        object.__setattr__(self, "theta", normalize_angles(self.theta))

    def __len__(self) -> int:
        """The number of particles."""
        return self.x.shape[0]


@dataclasses.dataclass
class RBSLAM(Inference):
    """Rao-blackwellized particle filter for SLAM."""

    # A 2x2 array for the motion covariance.
    motion_noise_covariance: np.ndarray

    # A 2x2 numpy array for the sensor noise for the
    # measurements of range and bearing to the landmarks
    sensor_noise_covariance: np.ndarray

    # A 2x2 array for the covariance when initializing new landmarks
    # By default, we simply use a large enough symmetric covariance
    new_landmarks_init_covariance: np.ndarray = np.eye(2) * 10

    num_particles: int = 10

    particles: RBSLAMParticles = None

    # To account for the approximation error of EKF when a landmark gets really
    # close, we will ignore any measurement that is within min_sensing_range!
    min_sensing_range: float = 1.

    def __post_init__(self):
        if self.particles:
            assert len(self.particles) == self.num_particles

    def estimated_pose(self) -> Pose:
        return Pose(np.mean(self.particles.x), np.mean(self.particles.y),
                    circular_mean(self.particles.theta))

    def estimated_landmarks(self) -> Sequence[Landmark]:
        landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        return [
            Landmark(idx_to_id[i], *loc) for i, loc in enumerate(landmarks_loc)
        ]

    def init(self, init_state: Pose) -> None:
        self.particles = RBSLAMParticles(
            np.full(self.num_particles, init_state.x, dtype=np.float64),
            np.full(self.num_particles, init_state.y, dtype=np.float64),
            np.full(self.num_particles, init_state.theta, dtype=np.float64),
            {},
            np.empty((self.num_particles, 0, 2), dtype=np.float64),
            np.empty((self.num_particles, 0, 2, 2), dtype=np.float64),
        )

    def get_measured_landmarks_idx(
            self, measurements: Sequence[Measurement]) -> np.ndarray:
        """Given a sequence of measurements, find the indices of each of the 
        landmark in `self.particles`.

        Args:
            measurements: a sequence of measures. This function assumes that all
            landmarks in this input sequence of measurements are already tracked 
            (i.e., using `self.add_new_landmarks`).
        """
        return np.array(
            [
                self.particles.landmarks_id_to_idx[m.landmark_id]
                for m in measurements
            ],
            dtype=int,
        )

    def plot_state(self, ax: 'matplotlib.axes.Axes') -> None:
        import matplotlib.pyplot as plt

        cmap = plt.get_cmap('viridis')

        ax.quiver(
            self.particles.x,
            self.particles.y,
            np.cos(self.particles.theta),
            np.sin(self.particles.theta),
            angles="xy",
            scale=100,
            color=(1.0, 0, 0, 0.1),
        )

        idx_to_id = {
            idx: id for id, idx in self.particles.landmarks_id_to_idx.items()
        }
        landmark_idx_to_color_code = np.array([
            float(hash(idx_to_id[idx]) % 256) / 256
            for idx in range(self.particles.landmarks_loc.shape[1])
        ])

        ax.scatter(self.particles.landmarks_loc[..., 0].flatten(),
                   self.particles.landmarks_loc[..., 1].flatten(),
                   marker="x",
                   c=np.broadcast_to(
                       landmark_idx_to_color_code,
                       self.particles.landmarks_loc.shape[:-1]).flatten(),
                   cmap=cmap)

        avg_landmarks_loc = np.mean(self.particles.landmarks_loc, axis=0)
        avg_landmarks_cov = np.mean(self.particles.landmarks_cov, axis=0)
        for i, (loc, cov) in enumerate(zip(avg_landmarks_loc,
                                           avg_landmarks_cov)):
            edgecolor = cmap(landmark_idx_to_color_code[i])
            confidence_ellipse(loc, cov, ax, n_std=5, edgecolor=edgecolor)

    # We will incrementally implement methods below using the `implementation_for`
    # helper
    def motion_model(self, command: Command) -> None:
        raise NotImplementedError()

    def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def update_EKF(self, measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()

    def compute_weights(self, measurements: Sequence[Measurement],
                        sensor_noise_covariance: np.ndarray) -> np.ndarray:
        raise NotImplementedError()

    def update(self, command: Command,
               measurements: Sequence[Measurement]) -> None:
        raise NotImplementedError()


def confidence_ellipse(loc, cov, ax, n_std=3.0, facecolor="none", **kwargs):
    """Draws a confidence ellipse for a 2D Gaussian."""

    from matplotlib.patches import Ellipse
    import matplotlib.transforms as transforms

    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
                      width=ell_radius_x * 2,
                      height=ell_radius_y * 2,
                      facecolor=facecolor,
                      **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std

    # calculating the stdandard deviation of y
    scale_y = np.sqrt(cov[1, 1]) * n_std

    transf = transforms.Affine2D()\
        .rotate_deg(45)\
        .scale(scale_x,scale_y)\
        .translate(*loc)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

### Question

Please implement a function that takes in a control command $[\Delta p_t, \Delta \theta_t]$ 
and updates the particles by drawing from the posterior $X_{t+1}$ using the 
motion model. 

This function behaves the same as `Localization.motion_model` for the pose 
components of the particles, and simply keeps the landmarks components 
(`landmarks_id_to_idx`, `landmarks_loc` and `landmarks_cov`) of the 
particles the same.


For reference, our solution is **9** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def motion_model(self: RBSLAM, command: Command) -> None:
    """A motion model that simulates a one-step movement of the robot and updates the particles.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
            movement, and delta_theta(float), the rotation of the movement.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rbslam_motion_model_problem_test():
    # A simulator with no noise
    sim = Simulator(motion_noise_covariance=np.zeros((2, 2)),
                    sensor_noise_covariance=np.zeros((2, 2)),
                    landmarks=[])
    sim.init()
    # Particle filter with implemented motion_model
    num_particles = 10
    pf = RBSLAM(
        motion_noise_covariance=[[.01, 0], [0, np.deg2rad(5)**2]],
        sensor_noise_covariance=sim.sensor_noise_covariance,
        num_particles=num_particles,
    )
    pf.init(sim.state)
    command = Command(1, 0)
    assert pf.motion_model(command) is None, "update should be in-place"
    assert len(pf.particles) == num_particles
    sim.simulate_motion(command)
    pose = sim.state
    est_pose = pf.estimated_pose()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=0.1
    ), "Estimated pose should be quite close to simulation without noise"

rbslam_motion_model_problem_test()

print('Tests passed.')

## RBSLAM: Tracking New Landmarks


### Question

Since our robot might have limited sensing range (see the parameter `Simulator.max_sensing_range`),
the robot is not aware of all the possible landmarks out there in the world.
Therefore, we must track new landmarks that appears in the measurements in our 
particles.

Please implement `add_new_landmarks` that takes a sequence of measurements and updates 
the particles to include potentially new landmarks that are not yet tracked by the 
particles. 

When you encounter a new landmark, instead of generating a set of particles to represent a distribution, 
you should store that landmark into each particle.
For each particle, the new landmark location's distribution 
- is centered the observed location of the landmark (relative to the particle's robot pose).
- has a fixed initial covariance of `self.new_landmarks_init_covariance`.


For reference, our solution is **31** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def add_new_landmarks(self, measurements: Sequence[Measurement]) -> None:
    """Add new landmarks into all particles to track them.

    Args:
        measurements: the measurements. This function will add only the new
        landmarks in these measurements (i.e., those that had not been added 
        before).
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rbslam_add_new_landmarks_test(add_new_landmarks, infer: RBSLAM,
                                  measurements: Sequence[Measurement],
                                  results: RBSLAMParticles):
    add_new_landmarks(infer, measurements)
    assert (np.allclose(infer.particles.x, results.x) and np.allclose(
        infer.particles.y,
        results.y)), "pose component of the particles should not change"
    assert infer.particles.landmarks_id_to_idx == results.landmarks_id_to_idx, "indexing mapping must be the same"
    assert np.allclose(
        infer.particles.landmarks_loc,
        results.landmarks_loc), "landmark locations should match exactly"
    assert np.allclose(
        infer.particles.landmarks_cov,
        results.landmarks_cov), "landmark covariances should match exactly"

rbslam_add_new_landmarks_test(add_new_landmarks, RBSLAM(motion_noise_covariance=np.zeros((2, 2), dtype=np.float64), sensor_noise_covariance=np.zeros((2, 2), dtype=np.float64), new_landmarks_init_covariance=np.array([[0.42, 0.  ],  [0.  , 0.42]], dtype=np.float64), num_particles=3, particles=RBSLAMParticles(x=np.array([0, 1, 2], dtype=np.int64), y=np.array([0, 1, 2], dtype=np.int64), theta=np.zeros((3,), dtype=np.float64), landmarks_id_to_idx={'Stata': 0, 'Building-4': 1}, landmarks_loc=np.array([[[ 2,  3],   [ 1,  1]],   [[ 1,  2],   [ 0,  0]],   [[ 0,  1],   [-1, -1]]], dtype=np.int64), landmarks_cov=np.zeros((3, 2, 2, 2), dtype=np.float64)), min_sensing_range=1.0), [Measurement(landmark_id='Stata', r=100, b=0.1), Measurement(landmark_id='Cafe-Luna', r=1, b=1.5707963267948966)], RBSLAMParticles(x=np.array([0, 1, 2], dtype=np.int64), y=np.array([0, 1, 2], dtype=np.int64), theta=np.zeros((3,), dtype=np.float64), landmarks_id_to_idx={'Stata': 0, 'Building-4': 1, 'Cafe-Luna': 2}, landmarks_loc=np.array([[[ 2.000000e+00,  3.000000e+00],   [ 1.000000e+00,  1.000000e+00],   [ 6.123234e-17,  1.000000e+00]],   [[ 1.000000e+00,  2.000000e+00],   [ 0.000000e+00,  0.000000e+00],   [ 1.000000e+00,  2.000000e+00]],   [[ 0.000000e+00,  1.000000e+00],   [-1.000000e+00, -1.000000e+00],   [ 2.000000e+00,  3.000000e+00]]], dtype=np.float64), landmarks_cov=np.array([[[[0.  , 0.  ],    [0.  , 0.  ]],    [[0.  , 0.  ],    [0.  , 0.  ]],    [[0.42, 0.  ],    [0.  , 0.42]]],    [[[0.  , 0.  ],    [0.  , 0.  ]],    [[0.  , 0.  ],    [0.  , 0.  ]],    [[0.42, 0.  ],    [0.  , 0.42]]],    [[[0.  , 0.  ],    [0.  , 0.  ]],    [[0.  , 0.  ],    [0.  , 0.  ]],    [[0.42, 0.  ],    [0.  , 0.42]]]], dtype=np.float64)))
print('Tests passed.')

## RBSLAM: Extended Kalman Filter Update


### Question

Please implement `update_EKF` that takes a sequence of measurements and performs
an extended Kalman filter update on each landmark of the particles.

One of the magical properties of Rao-Blackwellization for SLAM is that we can
    assume the landmarks are all independent of each other, given the
    pose. Since our particles are already conditioning the landmarks on the
    filtered pose, we can run a separate estimator for each landmark. This has
    very nice consequences for computational complexity --- our complexity
    grows linearly with the number of landmarks (a 2x2 EKF for each landmark),
    as opposed to quadratically for a single EKF over all landmarks (requiring
    a 2nx2n EKF). 

Suppose our robot's pose is $\texttt{Pose}(x, y, \theta)$.
The state that we would like to track by our filter is the landmark location $[\hat{l_x}, \hat{l_y}]$.
Recall that our observation model (without noise) is 
$$
\mathit{Obs}\left(\begin{bmatrix} l_x \\ l_y \end{bmatrix}\right) = \begin{bmatrix} 
\sqrt{(l_x - x)^2 + (l_y -y)^2} \\   
\texttt{atan2}(l_y - y, l_x - x) - \theta
\end{bmatrix}.
$$
This model is non-linear, and the standard Kalman filter does not apply.
This is where an extended Kalman filter could be useful.

In our case, an EKF update is the same as a Kalman filter update.
The idea of EKF is to approximate $\mathit{Obs}$ by its Jacobian around the current belief of location of the landmark, $[\hat{l_x}, \hat{l_y}]$
, so that we get a linear approximation:
$$
\mathit{Obs}\left(\begin{bmatrix} l_x \\ l_y \end{bmatrix}\right) \approx 
\nabla\mathit{Obs}\left(\begin{bmatrix} \hat{l_x} \\ \hat{l_y} \end{bmatrix}\right) \cdot \begin{bmatrix} l_x \\ l_y \end{bmatrix}
= \begin{bmatrix} 
\frac{\hat{l_x} - x}{r} & \frac{\hat{l_y} - y}{r} \\   
-\frac{\hat{l_y} - y}{r^2} & \frac{\hat{l_x} - x}{r^2}
\end{bmatrix} \cdot \begin{bmatrix} l_x \\ l_y \end{bmatrix} \triangleq H \cdot \begin{bmatrix} l_x \\ l_y \end{bmatrix} 
$$, where $r = \sqrt{(\hat{l_x} - x)^2 + (\hat{l_y} - y)^2}$.
Therefore, the implementation of an EKF for our application is the same as a KF 
where we use the matrix $H$ above for the observation model for each landmark. 


For reference, our solution is **46** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def update_EKF(self, measurements: Sequence[Measurement]) -> None:
    """Performs an extended Kalman filter update on the landmarks.

    Args:
        measurements: the sequence of landmark measurements.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rbslam_update_EKF_test(_, infer: RBSLAM,
                           measurements: Sequence[Measurement],
                           results: RBSLAMParticles):
    """Checks if estimation after one step update matchs our estimation."""
    infer.update_EKF(measurements)
    assert infer.particles.landmarks_id_to_idx == results.landmarks_id_to_idx
    assert np.allclose(infer.particles.landmarks_loc,
                       results.landmarks_loc,
                       atol=1e-3)
    assert np.allclose(infer.particles.landmarks_cov,
                       results.landmarks_cov,
                       atol=1e-3)

rbslam_update_EKF_test(update_EKF, RBSLAM(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 5.79948578e-09]], dtype=np.float64), new_landmarks_init_covariance=np.zeros((2, 2), dtype=np.float64), num_particles=10, particles=RBSLAMParticles(x=np.array([1.02110609, 1.0150698 , 1.00646616, 0.99664909, 0.98543836, 0.98580766, 0.99434219, 0.99586238, 1.01278529, 0.9728372 ], dtype=np.float64), y=np.zeros((10,), dtype=np.float64), theta=np.array([0.79952902, 0.79357213, 0.77358185, 0.80366545, 0.78089535, 0.79990768, 0.80084351, 0.78035684, 0.79478648, 0.78990957], dtype=np.float64), landmarks_id_to_idx={'L-only': 0}, landmarks_loc=np.array([[[ 0.82141275, 14.14085142]],   [[ 0.89961493, 14.14179007]],   [[ 1.17371392, 14.14127238]],   [[ 0.73846499, 14.13990443]],   [[ 1.04926045, 14.14211734]],   [[ 0.78075972, 14.14077479]],   [[ 0.77606104, 14.1405767 ]],   [[ 1.06730002, 14.14208092]],   [[ 0.88015748, 14.14163944]],   [[ 0.90917818, 14.14211808]]], dtype=np.float64), landmarks_cov=np.zeros((10, 1, 2, 2), dtype=np.float64)), min_sensing_range=1.0), [Measurement(landmark_id='L-only', r=14.142261353952044, b=0.7853881030367988)], RBSLAMParticles(x=np.array([1.02110609, 1.0150698 , 1.00646616, 0.99664909, 0.98543836, 0.98580766, 0.99434219, 0.99586238, 1.01278529, 0.9728372 ], dtype=np.float64), y=np.zeros((10,), dtype=np.float64), theta=np.array([0.79952902, 0.79357213, 0.77358185, 0.80366545, 0.78089535, 0.79990768, 0.80084351, 0.78035684, 0.79478648, 0.78990957], dtype=np.float64), landmarks_id_to_idx={'L-only': 0}, landmarks_loc=np.array([[[ 0.82141275, 14.14085142]],   [[ 0.89961493, 14.14179007]],   [[ 1.17371392, 14.14127238]],   [[ 0.73846499, 14.13990443]],   [[ 1.04926045, 14.14211734]],   [[ 0.78075972, 14.14077479]],   [[ 0.77606104, 14.1405767 ]],   [[ 1.06730002, 14.14208092]],   [[ 0.88015748, 14.14163944]],   [[ 0.90917818, 14.14211808]]], dtype=np.float64), landmarks_cov=np.zeros((10, 1, 2, 2), dtype=np.float64)))
print('Tests passed.')

## RBSLAM: Computing the Importance Weights


### Question

Please implement a function that takes a set of particles and measurements and
returns the importance weights of the particles. As in the pure particle
filter approach implementation, the weight is just the likelihood of the
    measurement given the sample, but remember that our sample is now a
    Gaussian for each landmark (the EKF). So the likelihood of each
    measurement is going to be given by the product of the Gaussian of the
    landmark and the sensor model. 


For reference, our solution is **56** line(s) of code.

In [ ]:
@implementation_for(RBSLAM)
def compute_weights(self: RBSLAM,
                    measurements: Sequence[Measurement]) -> np.ndarray:
    """Compute the importance weights of the particles, based on the new
    measurement.

    Args:
        measurements: a sequence of measurements made by the robot.

    Returns:
        weights: a numpy array of importance weights, normalized to 1.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rbslam_compute_weights_test(_, infer: RBSLAM,
                                measurements: Sequence[Measurement],
                                result_weights: np.ndarray):
    weights = infer.compute_weights(measurements)
    assert np.allclose(weights, result_weights, atol=1e-4)

rbslam_compute_weights_test(compute_weights, SLAM(motion_noise_covariance=np.array([[0.001     , 0.        ],  [0.        , 0.00274156]], dtype=np.float64), sensor_noise_covariance=np.array([[1.        , 0.        ],  [0.        , 0.00761544]], dtype=np.float64), new_landmarks_init_covariance=np.array([[10.,  0.],  [ 0., 10.]], dtype=np.float64), num_particles=20, particles=SLAMParticles(x=np.zeros((20,), dtype=np.float64), y=np.zeros((20,), dtype=np.float64), theta=np.zeros((20,), dtype=np.float64), landmarks_id_to_idx={'landmark-21': 0, 'landmark-24': 1, 'landmark-4': 2, 'landmark-1': 3, 'landmark-11': 4, 'landmark-26': 5, 'landmark-8': 6, 'landmark-19': 7, 'landmark-25': 8, 'landmark-29': 9, 'landmark-16': 10, 'landmark-23': 11, 'landmark-7': 12, 'landmark-28': 13, 'landmark-17': 14, 'landmark-30': 15, 'landmark-35': 16, 'landmark-13': 17, 'landmark-10': 18, 'landmark-12': 19, 'landmark-15': 20, 'landmark-20': 21, 'landmark-14': 22, 'landmark-9': 23, 'landmark-33': 24, 'landmark-32': 25, 'landmark-2': 26, 'landmark-5': 27, 'landmark-34': 28, 'landmark-22': 29, 'landmark-18': 30, 'landmark-6': 31, 'landmark-0': 32, 'landmark-31': 33, 'landmark-27': 34, 'landmark-3': 35}, landmarks_loc=np.array([[[ 1.00795109e+01,  8.65858550e+00],   [ 1.05658032e+01, -3.99709667e+00],   [-5.45004602e+00,  1.20335054e+01],   [-5.92592204e+00, -2.40457777e+00],   [ 2.20360566e+00,  1.77406287e+01],   [ 1.23088380e+01,  2.09325158e+00],   [ 1.09530711e+00,  5.05764451e+00],   [ 1.17854145e+01,  3.38730787e+00],   [ 1.31507227e+01, -3.50729613e+00],   [ 6.39454024e+00,  1.67344393e+01],   [-1.93805887e+00,  9.78302348e+00],   [ 6.68844248e+00,  1.41028148e+01],   [-1.33528167e-01, -4.19210727e+00],   [ 1.30910760e+01,  1.36664572e+01],   [ 1.85600524e+00,  1.79228336e+01],   [ 2.33697629e+01, -4.31900764e+00],   [ 1.71794549e+01,  1.72415309e+01],   [ 7.27885202e+00, -5.30416355e+00],   [-3.21405784e+00,  1.17991184e+01],   [ 7.91242721e+00, -3.51110402e+00],   [-2.76236365e+00,  4.66221097e+00],   [ 6.35579650e+00,  6.44835091e+00],   [ 1.89033675e+00,  1.91783804e+00],   [ 1.82021797e+00,  1.35426128e+01],   [ 1.51803235e+01,  1.21081824e+01],   [ 1.07362665e+01,  2.97064659e+00],   [-4.56099849e-01,  5.42727622e+00],   [-3.87424832e+00,  1.33852172e+01],   [ 1.35243112e+01,  1.26611411e+01],   [ 3.06053011e+00,  1.56653223e+01],   [ 8.96647583e+00, -4.88442391e+00],   [-2.20829631e+00, -5.95642765e+00],   [-1.80418969e+00, -9.37560410e-01],   [ 1.46091578e+01, -1.05702374e+00],   [ 2.04640508e+01,  9.39759023e+00],   [-2.25536739e+00,  7.35388364e+00]],   [[ 1.11036255e+01,  3.96880090e+00],   [ 1.74893775e+01, -4.20005376e+00],   [-1.69034143e+00,  1.34087634e+01],   [-5.70195894e+00, -9.92425360e-01],   [ 5.54870312e+00,  1.84491639e+01],   [ 5.79063420e+00, -1.50783431e+00],   [-2.96023379e+00, -1.15097472e+00],   [ 3.20321416e+00,  4.29180714e-01],   [ 9.98501506e+00,  3.49793520e+00],   [ 1.53072000e+01,  1.72837205e+01],   [ 7.84173615e+00,  1.23413031e+01],   [ 5.85148769e+00,  1.58388931e+01],   [ 3.89134945e+00, -5.09796177e+00],   [ 1.16994649e+01,  1.14557013e+01],   [ 4.73754263e+00,  1.44626956e+01],   [ 1.64951579e+01, -3.10490600e+00],   [ 1.32483702e+01,  2.35181098e+01],   [ 5.60261235e+00, -9.16609149e-01],   [-1.67960139e+00,  1.46007224e+01],   [ 5.43034669e+00,  4.13398262e+00],   [ 1.12296604e+00,  3.33723667e+00],   [ 8.78128768e+00,  5.77186229e+00],   [-2.39901359e+00,  3.09306245e+00],   [-2.77508698e+00,  8.15383930e+00],   [ 1.44942644e+01,  6.51697439e+00],   [ 1.90831675e+01,  4.97214502e-01],   [-7.45636909e+00,  1.35832305e+00],   [-5.68744800e+00,  1.49577865e+01],   [ 1.76831206e+01,  1.02132312e+01],   [ 6.45094631e+00,  6.31773483e+00],   [-3.54510936e+00, -4.06782590e+00],   [ 4.55837165e+00, -4.08302203e+00],   [-5.93050319e+00, -3.83094564e+00],   [ 1.37517720e+01,  1.63482212e+00],   [ 1.25395752e+01,  3.10684886e+00],   [-3.21311919e+00,  1.60138234e+01]],   [[ 1.34482304e+01,  7.76720283e+00],   [ 1.54413165e+01, -1.35881161e+00],   [-6.23896012e+00,  1.34014468e+01],   [-9.84271230e+00, -2.98444537e+00],   [ 2.44072091e+00,  1.38914203e+01],   [ 1.69493943e+01,  5.89330633e+00],   [-1.17613208e+00,  1.86560216e-01],   [ 7.33155651e+00, -3.76919171e+00],   [ 1.33348676e+01, -4.66525238e+00],   [ 1.26829283e+01,  1.68539306e+01],   [ 3.82929775e+00,  9.06265285e+00],   [ 8.00112769e+00,  2.32625359e+01],   [-5.42406546e-01,  1.01111858e+00],   [ 1.71201346e+01,  1.29239420e+01],   [ 4.95510992e+00,  1.24222267e+01],   [ 1.09093925e+01, -3.85970746e+00],   [ 1.36552677e+01,  1.65590219e+01],   [ 4.86297606e+00, -2.00836808e+00],   [ 1.69082390e+00,  1.72227163e+01],   [ 1.91578884e+00, -5.40869320e+00],   [ 2.34459648e+00,  1.08731550e+01],   [ 5.11720860e+00,  1.84995632e+00],   [-2.54198136e+00,  2.67819155e+00],   [ 5.40754703e-01,  1.26885083e+01],   [ 1.76166323e+01,  8.86412933e+00],   [ 1.89201001e+01,  2.98959348e+00],   [-1.02929453e+01, -1.22130650e-01],   [ 5.87107112e-01,  1.56331161e+01],   [ 1.12221656e+01,  1.58203688e+01],   [ 1.13231698e+01,  9.19690547e+00],   [ 9.26205853e+00, -1.63877263e+01],   [ 2.83041864e+00, -2.01782660e+00],   [-7.88366109e+00, -3.24400712e+00],   [ 1.63976484e+01,  1.69731783e+00],   [ 1.25485207e+01,  1.59027983e+00],   [ 1.48207723e+00,  6.81103249e+00]],   [[ 8.51288117e+00,  4.80311283e+00],   [ 9.36663858e+00, -2.78218131e+00],   [-2.52666718e+00,  1.41414833e+01],   [-1.79545531e+00, -1.10260286e+00],   [ 2.81024614e-01,  1.75647268e+01],   [ 9.37916133e+00,  6.60076496e+00],   [ 1.21625243e+00,  5.32717822e+00],   [ 8.86026820e+00, -3.76575917e+00],   [ 8.71400335e+00,  2.00015614e+00],   [ 1.67434372e+01,  1.31913270e+01],   [ 2.59157622e+00,  1.31554639e+01],   [ 2.40148430e+00,  1.71459865e+01],   [ 6.08959652e-01, -6.62248479e+00],   [ 1.74874680e+01,  7.78551796e+00],   [-4.66263790e-01,  1.69563981e+01],   [ 1.67124097e+01, -1.69993834e+00],   [ 1.38994982e+01,  1.65847751e+01],   [ 1.90161866e+00, -4.01252311e+00],   [ 3.54084368e+00,  1.10204815e+01],   [ 4.53107700e+00, -1.02460965e+01],   [ 1.93742020e+00,  4.60250331e+00],   [ 8.31670105e+00,  4.52493949e+00],   [ 2.88826989e+00,  7.03960760e+00],   [-1.96345699e-01,  1.28843475e+01],   [ 1.22977412e+01,  8.71234333e+00],   [ 1.65586615e+01,  1.05757603e+00],   [-4.46977579e+00,  2.33070323e-02],   [-6.45858464e+00,  9.42316476e+00],   [ 1.27551392e+01,  1.54226289e+01],   [ 7.28923612e+00,  1.53342137e+01],   [ 5.29904166e+00, -4.18395374e+00],   [-4.08494225e+00,  2.49005231e+00],   [-4.89188675e+00, -1.51251957e+00],   [ 1.87741176e+01,  1.75157580e-01],   [ 9.68343125e+00,  6.29713594e+00],   [-6.25662794e+00,  7.30057378e+00]],   [[ 1.02412146e+01,  8.24253799e+00],   [ 1.38380469e+01, -3.61244312e+00],   [-8.83111488e+00,  1.57799796e+01],   [-1.56752373e+00, -4.03073699e+00],   [ 3.28361888e+00,  1.31095895e+01],   [ 9.03296424e+00,  3.82871186e+00],   [ 5.55859074e+00,  5.02021949e+00],   [ 8.65418152e+00, -4.06984319e+00],   [ 1.95292556e+01,  2.44681442e+00],   [ 3.28512102e+00,  1.86376736e+01],   [ 9.72786301e-01,  1.35612087e+01],   [ 8.49335252e+00,  1.27294102e+01],   [ 6.67739750e+00,  6.39186540e-01],   [ 1.53359173e+01,  1.47887986e+01],   [ 1.25510882e+00,  1.27445285e+01],   [ 1.84972642e+01,  3.75863028e-01],   [ 1.07656330e+01,  2.29793986e+01],   [ 3.46790569e+00,  2.31829932e+00],   [ 1.91308516e+00,  8.27379148e+00],   [ 2.37524517e+00, -1.05127547e+01],   [-1.13877720e+00,  2.57870422e+00],   [ 8.99597147e+00,  9.54868021e+00],   [ 4.21427572e+00,  5.14229009e+00],   [ 8.50208166e-01,  1.14694020e+01],   [ 1.52671069e+01,  6.13701877e+00],   [ 1.53268763e+01,  2.54257632e+00],   [-6.97552504e+00,  4.90606647e+00],   [-5.11946769e+00,  1.65805865e+01],   [ 1.30295809e+01,  1.03119317e+01],   [ 1.35766765e+01,  1.06635391e+01],   [ 8.93091035e+00, -8.03400571e+00],   [-1.44131647e+00, -3.57678493e+00],   [ 6.23873240e-01, -6.42706245e+00],   [ 1.49278316e+01,  1.94985903e+00],   [ 9.07154991e+00,  5.70807814e+00],   [-4.70059330e+00,  1.00841796e+01]],   [[ 1.26550054e+01,  9.79712369e+00],   [ 1.49904908e+01, -3.92161150e+00],   [-5.32887733e+00,  5.05341028e+00],   [-8.08837148e+00,  2.49189330e-01],   [-3.51810623e+00,  1.59711792e+01],   [ 1.01273704e+01,  6.13566642e-01],   [-1.45436886e+00,  3.97927345e+00],   [ 9.54050391e+00, -6.55670822e+00],   [ 1.09786673e+01,  2.57379129e+00],   [ 9.89726197e+00,  1.77185290e+01],   [-2.21072972e+00,  1.07467310e+01],   [ 1.30095121e+01,  1.31419326e+01],   [-3.65223970e+00,  4.80083298e-01],   [ 1.48529788e+01,  1.42903823e+01],   [ 1.95218536e+00,  1.63313232e+01],   [ 1.66708829e+01, -1.97569172e+00],   [ 1.33897410e+01,  2.20354010e+01],   [ 2.03920854e+00,  2.36917287e+00],   [ 5.90435255e-02,  9.48849745e+00],   [ 3.07473100e+00, -5.77804897e+00],   [ 1.08834877e+00,  4.04470566e+00],   [ 1.10742810e+01,  2.99353961e+00],   [ 2.35524884e+00,  5.61611448e+00],   [-1.95146070e+00,  4.82351789e+00],   [ 1.35272913e+01,  1.01429536e+01],   [ 1.48677695e+01,  4.71671552e+00],   [-6.63639231e+00,  7.03121873e-01],   [-7.38945847e+00,  1.59001773e+01],   [ 1.70985190e+01,  7.93871525e+00],   [ 9.93379226e+00,  1.32952404e+01],   [ 1.83488068e+00, -6.74382545e+00],   [ 4.27025308e+00, -2.55242129e+00],   [-4.24863754e+00, -2.55094971e+00],   [ 2.41456719e+01,  3.61152815e+00],   [ 1.00315740e+01,  1.01196841e+01],   [-7.25005477e+00,  1.09353164e+01]],   [[ 1.02569267e+01,  1.08080066e+01],   [ 1.75183584e+01, -8.72256019e+00],   [-4.41132342e+00,  1.75191922e+01],   [-2.19830500e+00,  1.22355599e-01],   [-1.54983949e+00,  1.55405739e+01],   [ 8.51129922e+00, -7.99361481e-01],   [ 1.85496426e+00,  1.88117839e+00],   [ 4.29424128e+00,  8.17627140e-01],   [ 9.85793754e+00,  5.31129419e+00],   [ 1.06161186e+01,  2.23616157e+01],   [-5.21312415e-02,  1.42043111e+01],   [ 1.30567077e+01,  1.64912875e+01],   [ 4.49720272e+00, -5.48967594e+00],   [ 1.45952703e+01,  1.06113114e+01],   [ 6.00851677e+00,  1.83715920e+01],   [ 7.81459232e+00, -5.43147070e+00],   [ 1.30013484e+01,  1.71004746e+01],   [-1.92387285e+00, -3.31780664e+00],   [ 3.29668440e+00,  1.19048286e+01],   [ 1.66700122e+00, -4.72460086e+00],   [ 5.97452351e+00,  1.44948388e+00],   [ 8.20028192e+00,  4.49199143e+00],   [ 4.89139635e+00,  5.66382969e+00],   [ 4.43097836e+00,  1.10711819e+01],   [ 1.46294337e+01,  1.47425092e+01],   [ 1.39706928e+01,  3.72282607e+00],   [ 7.50261649e-01,  7.16821499e+00],   [-1.62894000e+00,  7.55747254e+00],   [ 1.52502492e+01,  1.13700064e+01],   [ 1.21562191e+01,  1.32309678e+01],   [ 8.00797537e+00, -2.04824449e+00],   [-2.30256922e+00, -3.72821051e+00],   [-4.02661402e+00, -9.45880676e+00],   [ 1.85548488e+01,  4.25777455e+00],   [ 1.76660189e+01,  7.47111698e+00],   [-1.31242684e+00,  2.31422154e+00]],   [[ 7.00078369e+00,  7.54446029e+00],   [ 1.16027635e+01, -4.36850796e+00],   [-6.01715235e+00,  1.36932030e+01],   [-8.42036234e+00, -4.20313751e-02],   [ 1.91704723e+00,  1.56582636e+01],   [ 8.39795364e+00,  8.26877607e+00],   [-6.83087472e-01, -2.87539245e+00],   [ 9.27423984e+00, -4.41843651e-01],   [ 1.11448212e+01, -6.19337891e-01],   [ 1.19632786e+01,  1.72060556e+01],   [ 7.21542343e-02,  7.77687207e+00],   [ 8.18601571e+00,  1.08330314e+01],   [ 4.44426650e+00, -3.00896112e+00],   [ 9.67239842e+00,  7.41018256e+00],   [-7.21061953e-01,  1.31846225e+01],   [ 1.40525792e+01, -1.13892059e+01],   [ 7.38765978e+00,  1.43555256e+01],   [ 7.00617866e+00, -3.98166735e+00],   [-1.13072442e-01,  1.58692635e+01],   [ 9.22619899e-01, -5.31006115e+00],   [ 1.51424664e-01,  1.00873540e+01],   [ 1.03357724e+01,  7.17146547e+00],   [ 1.21944794e+01,  9.22359223e+00],   [-1.92326191e+00,  8.68121735e+00],   [ 1.19056235e+01,  5.34385746e+00],   [ 1.65290602e+01, -1.76556789e+00],   [-2.24159335e+00,  1.25780999e+00],   [-6.03725964e+00,  9.70420717e+00],   [ 2.22425385e+01,  1.56270791e+01],   [ 5.72162122e+00,  1.10505727e+01],   [ 1.13662672e+01, -7.49453007e+00],   [ 5.78089291e+00, -8.99801785e+00],   [-4.88790507e+00, -9.35499421e+00],   [ 9.43940238e+00,  5.21573202e+00],   [ 1.41631244e+01,  3.20418180e-01],   [-9.68394071e+00,  1.14142723e+01]],   [[ 7.70539308e+00,  1.25217387e+01],   [ 1.46585646e+01, -4.29742226e+00],   [-4.08902872e+00,  9.60994635e+00],   [-4.97666835e+00, -6.30719699e-01],   [ 4.33417830e+00,  1.46863196e+01],   [ 1.39992322e+01,  6.75531747e+00],   [ 6.03600091e-01, -2.47730650e+00],   [ 6.23283163e+00, -6.45884597e+00],   [ 7.63570468e+00,  1.99361983e+00],   [ 1.22777855e+01,  1.73642775e+01],   [ 1.56179707e+00,  9.92653960e+00],   [ 5.60476225e+00,  1.86864159e+01],   [ 2.41083194e+00, -4.32875519e+00],   [ 1.29879418e+01,  1.09578234e+01],   [ 8.80293140e+00,  1.00699270e+01],   [ 1.24730086e+01, -9.49800439e-01],   [ 1.16070719e+01,  2.41632872e+01],   [ 4.84483581e-01,  2.22776870e-01],   [ 1.89777768e+00,  1.41315297e+01],   [ 7.36929311e+00, -5.38507471e+00],   [ 5.38929327e+00,  4.88028413e+00],   [ 7.17713163e+00,  6.66228063e+00],   [ 9.07381702e+00,  3.34919833e+00],   [-4.42556220e+00,  1.31456067e+01],   [ 1.24875643e+01,  1.65773624e+01],   [ 1.35740980e+01,  4.37033832e+00],   [-1.28868589e+00,  6.24394186e+00],   [-3.47598540e+00,  1.47708404e+01],   [ 1.03418314e+01,  1.07582809e+01],   [ 1.02581455e+01,  1.26703614e+01],   [ 3.90021626e+00, -3.46324556e+00],   [ 5.48690450e-01, -1.97102703e-01],   [-5.65013946e+00, -5.77997251e+00],   [ 1.45466963e+01, -1.13845147e+00],   [ 1.50778015e+01,  3.57604577e+00],   [-1.09739217e+00,  5.80862775e+00]],   [[ 1.30325369e+01,  9.31608162e+00],   [ 1.34625812e+01, -7.72490660e+00],   [-3.15051465e+00,  1.15091965e+01],   [-3.28707449e+00, -2.05330925e+00],   [-2.91106324e+00,  1.60734212e+01],   [ 1.18369672e+01,  4.61527791e+00],   [ 3.21212358e+00, -1.29674065e+00],   [-7.91451718e-01,  1.07085858e+00],   [ 1.27349854e+01, -1.82475024e+00],   [ 1.12522760e+01,  1.40610510e+01],   [-4.14220571e+00,  6.91127464e+00],   [ 7.36037413e+00,  2.00065368e+01],   [-3.28325521e-01,  2.58130977e+00],   [ 1.04796711e+01,  1.03399084e+01],   [ 6.10831429e+00,  1.60489454e+01],   [ 1.32365443e+01, -6.76500200e+00],   [ 1.48275484e+01,  2.03574243e+01],   [-7.54213376e-01,  2.84812703e+00],   [ 1.02530349e+00,  8.28084353e+00],   [ 5.04950246e+00, -4.56907397e+00],   [-4.95539030e-01,  5.56086123e+00],   [ 4.89614863e+00, -1.33248239e+00],   [ 4.77836714e+00,  7.37370479e+00],   [ 3.55504819e+00,  5.80548444e+00],   [ 2.32777781e+01,  1.55911278e+01],   [ 2.11874640e+01, -1.41013921e-01],   [-5.99713661e+00, -2.36817816e+00],   [-3.58933218e+00,  1.33695324e+01],   [ 1.60812742e+01,  1.05884717e+01],   [ 1.00944210e+01,  1.16578600e+01],   [ 3.56425027e+00, -4.70559999e+00],   [ 6.51949923e-01,  6.17255621e-01],   [-4.02415318e+00, -7.10987384e+00],   [ 1.64667678e+01,  1.92280724e+00],   [ 1.34696452e+01, -1.98774051e-02],   [-5.54444238e+00,  5.59988633e+00]],   [[ 7.35780231e+00,  1.47021898e+01],   [ 1.00135664e+01,  4.60236657e-01],   [-5.21068262e-01,  1.35266532e+01],   [-5.27605918e+00,  1.63150544e+00],   [ 2.62946776e-01,  1.34750386e+01],   [ 9.02997482e+00,  4.71017542e+00],   [-6.31374288e-01,  9.17029428e-01],   [ 1.12314614e+01, -2.65991917e+00],   [ 1.37768053e+01, -1.96664579e+00],   [ 5.67783436e+00,  1.28651736e+01],   [ 9.63749597e-01,  1.42557370e+01],   [ 9.25592322e+00,  1.48729029e+01],   [ 2.85906093e+00,  2.48577225e-01],   [ 1.15322089e+01,  1.32181196e+01],   [ 7.34581845e+00,  1.88407544e+01],   [ 1.39790642e+01, -2.23342652e+00],   [ 8.38688203e+00,  1.72361450e+01],   [ 5.35422227e+00,  6.11272338e-01],   [-5.74555150e-01,  1.54715007e+01],   [ 4.81276180e+00, -4.36511493e+00],   [ 1.73957026e+00,  1.05368332e+01],   [ 7.94403937e+00,  2.57980600e+00],   [ 3.95619082e+00,  9.47100420e+00],   [-2.56090804e+00,  1.12672129e+01],   [ 1.56077498e+01,  9.88937189e+00],   [ 1.49125045e+01, -1.50280979e+00],   [-4.56630084e+00,  6.03340278e+00],   [ 7.90529029e-01,  1.60115282e+01],   [ 1.29865056e+01,  1.55849419e+01],   [ 9.40238407e+00,  1.49946165e+01],   [ 1.37902136e+01, -7.19562593e-01],   [-1.49031199e+00, -6.23752227e+00],   [-6.65309349e-01, -5.74150614e+00],   [ 1.70226317e+01,  4.02750448e+00],   [ 1.72776642e+01,  6.50827377e+00],   [-1.36428550e+01,  8.86549097e+00]],   [[ 7.19483043e+00,  4.90898456e+00],   [ 1.27218067e+01, -5.43713307e-01],   [-7.76425061e+00,  1.63930213e+01],   [-2.19087885e+00,  2.51621651e+00],   [-8.90616747e-01,  1.44085957e+01],   [ 1.01842909e+01,  3.84510075e+00],   [-1.64730315e-01,  6.71118654e-01],   [ 6.17764786e+00, -1.10051090e+00],   [ 1.22972355e+01, -5.53966986e+00],   [ 1.22514930e+01,  2.27416692e+01],   [ 5.90972015e+00,  9.27354103e+00],   [ 1.05297571e+01,  2.28744616e+01],   [ 1.36409536e-01,  2.28751523e+00],   [ 1.96303931e+01,  1.37733029e+01],   [ 8.49317722e+00,  1.21643863e+01],   [ 1.56945802e+01, -2.78916053e+00],   [ 1.69045806e+01,  1.91159313e+01],   [ 4.70650145e+00, -4.01947010e+00],   [ 3.50018237e+00,  1.25435072e+01],   [ 1.38049226e+00, -1.52370171e+00],   [ 4.25015549e+00,  8.96495718e+00],   [ 7.96182533e+00,  1.03866948e+01],   [ 6.10649274e+00,  1.57268506e+00],   [-6.48868403e+00,  6.89345457e+00],   [ 8.99380641e+00,  9.94055959e+00],   [ 1.70938387e+01, -1.47549523e+00],   [-5.94398493e+00,  8.67335723e+00],   [-3.85454577e+00,  1.94693968e+01],   [ 1.49847794e+01,  1.26684134e+01],   [ 1.01196582e+00,  1.28788174e+01],   [ 1.06625559e+01, -5.64315238e+00],   [-1.22956963e+00, -3.16294747e+00],   [-6.65541466e+00, -6.60996620e+00],   [ 1.84696319e+01,  1.04409418e-01],   [ 1.58244453e+01,  2.89309380e+00],   [ 2.77582162e-01,  9.86824200e+00]],   [[ 8.38985634e+00,  5.23245510e+00],   [ 9.30058892e+00, -6.92424468e+00],   [-8.72028959e+00,  1.56040562e+01],   [-4.20415956e+00,  2.94775580e+00],   [ 7.18512455e+00,  1.93349451e+01],   [ 1.28167342e+01,  2.54416690e+00],   [ 2.40147178e+00,  2.34234018e+00],   [ 5.91605226e+00, -2.06050899e+00],   [ 1.48641604e+01,  4.35598182e+00],   [ 9.84282695e+00,  1.59924851e+01],   [ 4.72598125e+00,  1.02561266e+01],   [ 9.13540974e+00,  2.11266762e+01],   [-4.17868726e-01,  8.49297418e-02],   [ 1.86510758e+01,  9.33516021e+00],   [ 4.03536541e+00,  1.26495116e+01],   [ 1.27269235e+01, -8.38894102e+00],   [ 2.43705995e+01,  1.84273278e+01],   [ 4.06756133e+00, -3.21204623e+00],   [-5.64185317e+00,  1.21098892e+01],   [ 4.64859494e+00, -2.33641705e+00],   [ 4.69081518e+00,  1.04677996e+01],   [ 8.73550216e+00,  7.18522017e+00],   [ 3.01137314e+00,  1.37822921e+00],   [-2.67948445e+00,  1.17516615e+01],   [ 1.84051272e+01,  1.07246359e+01],   [ 1.77056467e+01,  7.42686908e+00],   [-9.91886207e+00, -2.07426967e-01],   [-4.23504713e+00,  1.68289385e+01],   [ 1.48888934e+01,  1.60832659e+01],   [-1.54373372e+00,  3.20301932e+00],   [ 5.38673347e+00,  3.87432349e+00],   [-2.48517665e-01, -1.28220782e+01],   [-5.46555280e+00, -2.18335237e+00],   [ 2.05096937e+01,  3.50955082e+00],   [ 1.31464323e+01,  7.94988825e+00],   [-8.47843642e-01,  6.03810240e+00]],   [[ 7.79075670e+00,  7.56473900e+00],   [ 7.25210179e+00, -8.06222000e+00],   [-1.15234011e+01,  1.16319433e+01],   [-6.22722066e+00, -1.50998434e+00],   [ 1.57692383e+00,  1.68451675e+01],   [ 9.26802658e+00,  7.55530710e+00],   [-5.00309320e+00,  1.63150137e+00],   [ 6.42930779e+00,  2.25623859e+00],   [ 9.88609502e+00,  3.06345797e+00],   [ 9.97324660e+00,  1.70524888e+01],   [ 2.04675130e+00,  1.44723048e+01],   [ 9.92430782e+00,  1.37216642e+01],   [-3.55186464e+00, -3.09510363e+00],   [ 1.28814686e+01,  1.04625832e+01],   [ 4.03503634e+00,  1.33345980e+01],   [ 1.74658351e+01, -4.58662333e+00],   [ 9.95477453e+00,  1.71105484e+01],   [ 5.94374556e+00,  5.85294582e-01],   [ 5.40127832e-01,  1.20638264e+01],   [ 1.77542122e+00,  9.19971134e-01],   [ 1.46901579e+00,  1.19052765e+01],   [ 7.11656856e+00,  8.27514631e+00],   [ 5.05963027e+00,  4.38796123e+00],   [-4.57877431e+00,  1.31331051e+01],   [ 1.78121190e+01,  1.04791264e+01],   [ 2.05040956e+01,  2.62359654e+00],   [ 2.76887800e+00,  6.97646155e+00],   [-5.23962469e+00,  1.89097949e+01],   [ 1.57620919e+01,  1.95706351e+01],   [ 8.22484418e+00,  1.03221219e+01],   [ 7.47943898e+00, -3.29713850e+00],   [ 4.65105087e+00, -4.80789935e+00],   [-4.66450104e+00, -5.69649708e+00],   [ 1.99514218e+01,  2.68551290e-01],   [ 1.01959446e+01,  2.82372316e+00],   [-5.96613435e+00,  7.39882553e+00]],   [[ 1.15120391e+01,  6.16971334e+00],   [ 1.64311530e+01, -9.26581406e+00],   [ 4.14008049e+00,  6.34853041e+00],   [-4.65948897e+00, -1.12591668e+00],   [-7.97665161e-01,  1.73420862e+01],   [ 1.47381258e+01,  3.29952225e+00],   [ 3.85638943e+00,  1.87723214e+00],   [ 3.37633591e+00, -6.81721575e-01],   [ 9.78328825e+00,  1.13916114e+00],   [ 9.45052058e+00,  1.71398370e+01],   [-9.14026909e-01,  1.56463328e+01],   [ 6.10182062e+00,  1.22520390e+01],   [ 9.22489217e+00, -4.46080520e+00],   [ 1.39637494e+01,  3.67311076e+00],   [ 4.56500271e+00,  1.62226608e+01],   [ 1.41513924e+01, -8.34058043e+00],   [ 1.24052348e+01,  1.40687201e+01],   [ 4.99727635e-01, -2.86729963e+00],   [ 4.66659948e+00,  1.25825571e+01],   [ 9.31605185e+00, -7.05085053e+00],   [ 1.13757856e+00,  1.77367183e+00],   [ 5.10432633e+00,  4.29766977e+00],   [ 2.97004146e+00,  9.69122528e+00],   [-1.26838179e+00,  8.48165255e+00],   [ 1.79401615e+01,  1.08495196e+01],   [ 1.32527446e+01,  3.61088252e-01],   [ 2.04749245e-01,  7.88438492e+00],   [-3.38038127e+00,  1.78381095e+01],   [ 2.07646661e+01,  1.18305901e+01],   [ 8.54947246e+00,  1.12717547e+01],   [ 4.56767117e+00, -3.14874880e+00],   [-3.86492896e+00, -5.41061978e+00],   [-5.17741921e+00, -2.77769223e+00],   [ 1.31824321e+01,  3.73683282e-01],   [ 1.98471985e+01,  5.37694737e+00],   [-4.28041342e+00,  5.67495093e+00]],   [[ 8.32509210e+00,  9.19606030e+00],   [ 1.11416351e+01, -4.87590863e+00],   [-4.71765697e+00,  1.23004472e+01],   [-1.71088547e+00, -2.55446196e+00],   [ 4.23445268e+00,  1.77474808e+01],   [ 1.22007717e+01,  7.26450572e+00],   [ 3.45260208e-01,  2.01560897e+00],   [ 6.19148250e+00, -1.28627407e+00],   [ 1.07857237e+01, -6.20059897e+00],   [ 1.26110291e+01,  1.63553606e+01],   [-2.12678706e+00,  1.53331582e+01],   [ 4.14148102e+00,  1.83519753e+01],   [-1.44156611e+00,  3.03968090e+00],   [ 1.83640294e+01,  5.47119107e+00],   [ 2.78310681e+00,  1.86534880e+01],   [ 1.58339362e+01, -3.13787308e+00],   [ 1.80155403e+01,  1.41246941e+01],   [ 1.55281821e-01, -2.12658028e+00],   [ 4.34539515e+00,  8.92976130e+00],   [ 2.53641587e-01, -2.65794517e-01],   [ 8.81819318e-01,  5.84681447e+00],   [ 4.46106649e+00,  6.89548682e+00],   [ 6.09618180e+00,  2.33004488e+00],   [-5.38496852e+00,  3.33662638e+00],   [ 1.82835713e+01,  1.89753612e+01],   [ 1.79646220e+01,  4.26716422e+00],   [-3.75567359e+00,  4.86690449e+00],   [-4.93069350e+00,  1.55445203e+01],   [ 1.30366411e+01,  9.30403276e+00],   [ 8.31838528e+00,  1.08830201e+01],   [ 3.46843527e+00, -6.30426882e+00],   [-1.35532963e+00, -3.47048813e+00],   [-3.82528797e+00, -4.66606909e+00],   [ 1.24318507e+01,  2.87665778e+00],   [ 1.55655856e+01,  1.27544101e+01],   [-3.08447392e+00,  1.07146162e+01]],   [[ 1.11341105e+01,  2.26285014e+00],   [ 1.22514237e+01, -4.62339773e+00],   [-5.64581645e+00,  1.40738410e+01],   [-1.41752411e-01, -3.36129789e+00],   [ 1.19242108e+00,  1.65470543e+01],   [ 8.92976308e+00, -2.84905253e+00],   [-6.48110465e-01, -1.81677745e+00],   [ 6.10358088e-01,  2.42074014e+00],   [ 1.66232232e+01,  1.35644591e+00],   [ 1.20281906e+01,  1.87471159e+01],   [ 2.27275675e+00,  1.36851278e+01],   [ 6.81120335e+00,  1.32593576e+01],   [ 4.94372947e+00,  2.37520403e+00],   [ 2.07621491e+01,  7.92655853e+00],   [-9.80601126e-01,  1.98708214e+01],   [ 1.46168431e+01,  2.61862325e-01],   [ 2.14890198e+01,  1.22786531e+01],   [ 1.50120588e+00, -5.18601635e+00],   [-3.47901822e-01,  9.06524103e+00],   [-1.03000385e+00, -3.60792970e+00],   [ 6.55500191e+00,  4.19904441e+00],   [ 1.28071043e+01,  7.42195115e+00],   [ 4.09598412e+00,  7.25554310e-01],   [-3.09386618e+00,  1.60505495e+01],   [ 1.52563257e+01,  1.25657964e+01],   [ 1.58186296e+01, -1.13001773e+00],   [-2.03805198e+00,  8.97682708e+00],   [-8.25585611e-01,  1.97024611e+01],   [ 1.60219056e+01,  1.32193632e+01],   [ 9.63528995e+00,  9.86861363e+00],   [ 5.62807303e+00, -9.60065263e+00],   [ 2.97207698e+00, -5.57687580e+00],   [-9.39252935e+00, -9.88635903e+00],   [ 2.30693497e+01,  5.42290075e+00],   [ 1.51045367e+01,  1.16431473e+00],   [-5.33321762e+00,  4.52860116e+00]],   [[ 7.02917172e+00,  7.30930126e+00],   [ 1.03519572e+01, -8.49441805e+00],   [-1.85527377e+00,  1.21537476e+01],   [-3.55122215e+00,  3.15099497e+00],   [ 5.17354397e+00,  2.04626899e+01],   [ 1.43227671e+01,  5.29587464e+00],   [-1.27197226e+00, -2.40382470e+00],   [ 8.60686641e+00, -1.35488544e+00],   [ 1.77684263e+01, -1.91170234e+00],   [ 1.15984831e+01,  1.82746119e+01],   [-2.14059990e+00,  9.84528101e+00],   [ 9.44045037e-01,  1.65020792e+01],   [ 1.49377618e+00,  2.35939709e+00],   [ 1.00658658e+01,  7.16470797e+00],   [ 2.57123273e+00,  1.69346238e+01],   [ 1.15247535e+01, -2.82116414e+00],   [ 1.66704644e+01,  1.92133578e+01],   [ 3.08222686e+00,  4.06238321e-01],   [ 4.01319704e+00,  9.13195079e+00],   [ 4.98666834e+00,  2.41975524e+00],   [ 3.46295913e+00,  1.02495983e+01],   [ 9.81907394e+00,  8.40245471e+00],   [ 5.83280918e+00,  3.37687276e+00],   [-2.36990099e+00,  8.59764574e+00],   [ 1.85673227e+01,  1.08415935e+01],   [ 1.66331096e+01,  2.59787134e+00],   [-3.35109855e+00,  8.70486924e+00],   [-7.12393305e+00,  1.80249560e+01],   [ 1.42218202e+01,  1.21679225e+01],   [ 7.49767548e+00,  6.28903118e+00],   [ 5.17473109e+00, -4.99015476e+00],   [-2.13545655e+00, -1.63226201e+00],   [-6.93322131e+00, -4.66493854e+00],   [ 1.55567591e+01,  5.97847830e+00],   [ 9.64559353e+00,  7.11650633e+00],   [-7.11069718e-01,  1.22438719e+01]],   [[ 8.74098327e+00,  1.34605831e+01],   [ 9.63601280e+00, -5.83117069e+00],   [-5.89611044e+00,  1.21788068e+01],   [-1.19707350e+01,  2.56165622e+00],   [ 5.57935490e+00,  1.42917683e+01],   [ 1.53736891e+01,  5.65839895e+00],   [-3.05717032e+00, -5.10563296e-01],   [ 5.13610712e+00,  6.08299814e-01],   [ 1.44257754e+01,  1.80890702e+00],   [ 8.75131350e+00,  1.77312867e+01],   [ 5.79053966e-01,  1.72683099e+01],   [ 6.01071644e+00,  1.54209641e+01],   [ 7.21863600e-03, -3.78579671e+00],   [ 1.21091843e+01,  1.31142127e+01],   [ 3.04940834e+00,  1.04067761e+01],   [ 1.42897494e+01, -6.27688771e+00],   [ 1.52856833e+01,  1.96720245e+01],   [ 1.99837893e+00, -3.61073474e-01],   [ 2.74940610e-01,  6.83073954e+00],   [ 6.22368003e+00, -4.85406848e+00],   [-5.22644911e-01,  3.01099609e+00],   [ 8.73448708e+00,  7.81290572e+00],   [ 5.85277860e+00,  3.45442216e-01],   [-2.68864222e+00,  1.07704459e+01],   [ 2.02592053e+01,  1.55320211e+01],   [ 1.73688710e+01,  5.74672560e+00],   [-8.64429298e+00,  5.36985252e+00],   [ 8.41861472e-01,  1.44831609e+01],   [ 1.99207565e+01,  1.75142461e+01],   [ 9.94406960e+00,  1.58710056e+01],   [ 6.26460438e+00, -5.28463665e+00],   [ 4.57305383e+00, -4.50325209e+00],   [-8.25757383e+00, -4.05034499e+00],   [ 2.68903915e+01, -5.46926973e+00],   [ 1.80732557e+01,  4.19479564e+00],   [-2.82483407e+00,  1.04018860e+01]],   [[ 3.91750150e+00,  3.06789656e+00],   [ 9.89474428e+00, -4.39632403e+00],   [-9.51180089e+00,  1.34405183e+01],   [-3.41309416e+00,  5.05550122e-01],   [ 2.39654955e+00,  1.10324360e+01],   [ 1.03959435e+01,  3.97581842e+00],   [ 2.47805533e+00,  2.08886445e+00],   [ 1.20123239e+01,  1.54336711e+00],   [ 1.16239155e+01,  1.11444683e+00],   [ 1.58192316e+01,  1.38110412e+01],   [ 1.37948872e+00,  1.11589106e+01],   [ 5.10529177e+00,  1.26090510e+01],   [ 2.79428957e+00, -1.43703537e+00],   [ 1.42915689e+01,  9.05913953e+00],   [-4.01487049e-01,  1.94163518e+01],   [ 2.06365600e+01, -3.31817875e+00],   [ 1.67742891e+01,  2.21114288e+01],   [ 5.82113612e+00,  3.81486088e+00],   [ 2.38341738e+00,  1.55195791e+01],   [ 6.09141966e+00, -2.97855005e+00],   [ 4.42627123e+00,  2.99906454e+00],   [ 6.10720915e+00,  9.52224260e+00],   [ 8.79262034e+00,  5.43170174e+00],   [-4.62051707e+00,  1.31234579e+01],   [ 1.67399623e+01,  7.06383146e+00],   [ 1.86452071e+01,  5.16577940e+00],   [-1.44862270e+00,  2.78246540e+00],   [-5.83696364e+00,  1.04815232e+01],   [ 2.03750407e+01,  1.39005762e+01],   [ 1.29203190e+01,  9.75387310e+00],   [ 1.25944885e+01, -1.03184160e+01],   [ 2.68713543e-01, -8.30529604e+00],   [-6.24133808e+00, -8.33403835e+00],   [ 1.54298299e+01,  8.05796396e+00],   [ 1.21100239e+01,  9.22644680e+00],   [-1.06683882e+01,  1.55121249e+01]]], dtype=np.float64))), [Measurement(landmark_id='landmark-21', r=11.439438720078154, b=0.7738698392857971), Measurement(landmark_id='landmark-24', r=13.2895332911168, b=-0.31259629224640056), Measurement(landmark_id='landmark-4', r=12.113441267512407, b=1.92410200252704), Measurement(landmark_id='landmark-1', r=5.304000045130137, b=3.2242410590376145), Measurement(landmark_id='landmark-11', r=15.296264764193008, b=1.460367471318978), Measurement(landmark_id='landmark-26', r=12.025836178136165, b=0.32535692642774505), Measurement(landmark_id='landmark-8', r=1.6749692253611657, b=1.5517031522371973), Measurement(landmark_id='landmark-19', r=6.754089052746934, b=-0.06390238172222208), Measurement(landmark_id='landmark-25', r=11.45574101714269, b=-0.027602395766073297), Measurement(landmark_id='landmark-29', r=20.411630536374133, b=1.018271671965786), Measurement(landmark_id='landmark-16', r=12.520575977729484, b=1.3682922057448574), Measurement(landmark_id='landmark-23', r=17.223349146511705, b=1.1378237581792867), Measurement(landmark_id='landmark-7', r=0.9034701816518086, b=0.00820412066646277), Measurement(landmark_id='landmark-28', r=16.227063499123336, b=0.7049624503229832), Measurement(landmark_id='landmark-17', r=16.034696676803303, b=1.3450333131749717), Measurement(landmark_id='landmark-30', r=15.482804318931906, b=-0.2632326756081807), Measurement(landmark_id='landmark-35', r=22.468191988055043, b=0.832595844385654), Measurement(landmark_id='landmark-13', r=4.214659122506341, b=0.031012119222392227), Measurement(landmark_id='landmark-10', r=11.346171390581661, b=1.5594854034778332), Measurement(landmark_id='landmark-12', r=6.44082971955371, b=-0.6550717102177889), Measurement(landmark_id='landmark-15', r=7.685206377895039, b=1.2392634902611783), Measurement(landmark_id='landmark-20', r=10.290147333781464, b=0.531829891128762), Measurement(landmark_id='landmark-14', r=5.921309879821684, b=0.7580032298435072), Measurement(landmark_id='landmark-9', r=9.458020683536958, b=1.7418611358531606), Measurement(landmark_id='landmark-33', r=19.690178689864442, b=0.5784120625007443), Measurement(landmark_id='landmark-32', r=16.849802913129597, b=0.13953297039427145), Measurement(landmark_id='landmark-2', r=5.652400116372298, b=2.413482735561835), Measurement(landmark_id='landmark-5', r=15.204061038721088, b=1.8502558944089569), Measurement(landmark_id='landmark-34', r=20.42986369482223, b=0.7042422951317948), Measurement(landmark_id='landmark-22', r=13.23808713509877, b=0.9250492804923485), Measurement(landmark_id='landmark-18', r=8.507836662855938, b=-0.5657320834345437), Measurement(landmark_id='landmark-6', r=5.7393678771301335, b=-1.6140727018312917), Measurement(landmark_id='landmark-0', r=5.985823878952583, b=-2.3787592015612584), Measurement(landmark_id='landmark-31', r=17.583472878802123, b=0.11522323269730204), Measurement(landmark_id='landmark-27', r=15.055557724680874, b=0.39571009096001086), Measurement(landmark_id='landmark-3', r=8.996300884259046, b=2.0941068111930456)], np.array([5.84368144e-096, 3.64126381e-321, 1.33028775e-313, 9.36301325e-200, 6.55058743e-092, 3.24193326e-307, 4.52442068e-230, 4.57941643e-305, 1.36852156e-303, 0.00000000e+000, 1.00000000e+000, 2.18388990e-112, 0.00000000e+000, 3.53186960e-253, 1.38502630e-132, 3.69427451e-188, 0.00000000e+000, 3.48915715e-258, 5.04724011e-185, 1.15576838e-050], dtype=np.float64))
print('Tests passed.')

## RBSLAM: Particle Filter Update


### Question

Please implement the update step of RB SLAM.

The procedure is similar to `SLAM.update`, except it does an additional `update_EKF` step after `add_new_landmarks` and before `compute_weights`. 

Further, since by using EKF we approximated the non-linear observation model by a linear model, the approximation can be quite bad
when the robot is very close to a landmark. 
To fix, we will simply ignore any landmark measurement that is within `min_sensing_range`!
Don't panic! If everything is implemented correctly, you will see that even when we ignore information of nearby landmarks, 
we still get a much better result than using a plain particle filter for SLAM. 


For reference, our solution is **27** line(s) of code.

In addition to all the utilities defined at the top of the Colab notebook, the following functions are available in this question environment: `add_new_landmarks`, `compute_weights`, `motion_model`, `update_EKF`. You may not need to use all of them.

In [ ]:
@implementation_for(RBSLAM)
def update(self: RBSLAM, command: Command,
           measurements: Sequence[Measurement]) -> None:
    """Update the samples, based on the command and the new measurement.

    Args:
        command: a Command tuple containing fields delta_p(float), the distance of the
        movement, and delta_theta(float), the rotation of the movement.
        measurement: a measurement vector. The measurement is computed by
            `simulator.simulate_sensing` function.
    """
    # Filter out any measurement that gets too close,
    # since they mess up with our EKF updates
    measurements = [m for m in measurements if m.r > self.min_sensing_range]

    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rbslam_update_test(_, infer: RBSLAM, command: Command,
                       measurements: Sequence[Measurement], pose: Pose,
                       landmarks: Sequence[Landmark]):
    """Checks if estimation after one step update matchs our estimation."""
    infer.update(command, measurements)
    est_pose = infer.estimated_pose()
    est_landmarks = infer.estimated_landmarks()
    assert np.allclose(
        [pose.x, pose.y, pose.theta],
        [est_pose.x, est_pose.y, est_pose.theta],
        atol=1.0), "Estimated pose should be quite close to our estimation"

    landmarks = {l.id: l for l in landmarks}
    est_landmarks = {l.id: l for l in est_landmarks}

    assert landmarks.keys() == est_landmarks.keys()

    for lid in landmarks:
        assert np.allclose(
            [landmarks[lid].x, landmarks[lid].y],
            [est_landmarks[lid].x, est_landmarks[lid].y],
            atol=1.0
        ), "Estimated landmark location should be quite close to our estimation"

rbslam_update_test(update, RBSLAM(motion_noise_covariance=np.array([[0.0001, 0.    ],  [0.    , 0.0001]], dtype=np.float64), sensor_noise_covariance=np.array([[1.00000000e-06, 0.00000000e+00],  [0.00000000e+00, 5.79948578e-09]], dtype=np.float64), new_landmarks_init_covariance=np.zeros((2, 2), dtype=np.float64), num_particles=100, particles=RBSLAMParticles(x=np.zeros((100,), dtype=np.float64), y=np.zeros((100,), dtype=np.float64), theta=np.zeros((100,), dtype=np.float64), landmarks_id_to_idx={}, landmarks_loc=np.array([], dtype=np.float64).reshape((100, 0, 2)), landmarks_cov=np.array([], dtype=np.float64).reshape((100, 0, 2, 2))), min_sensing_range=1.0), Command(delta_p=1, delta_theta=0.7853981633974483), [Measurement(landmark_id='L-only', r=1.4143392925941884, b=0.7853881030367988)], Pose(x=1.0004415422399586, y=-0.00022564496898136023, theta=0.7868640441016728), [Landmark(id='L-only', x=0.9981907021298546, y=1.4142670083075757)])
print('Tests passed.')